In [1]:
import re
import json
import numpy as np
import pandas as pd
import jieba
from zhconv import convert
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA

In [2]:
# 读取20220831的原始cv和jd特征数据
all_data = pd.read_csv('../data_20220831/raw_cvjd_20220831_spark.csv').drop(['Unnamed: 0'], axis=1)

/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_74508/2442730307.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data = pd.read_csv('../data_20220831/raw_cvjd_20220831_spark.csv').drop(['Unnamed: 0'], axis=1)


# tfidf_pca

In [3]:
# 读取生成的5份tfidf_pca文件，并拼接
sample = all_data

# 需要将这个向量变成一个列表
def tfidf_pca_merge_fun(series):
    return list(series)

tfidf_pca_file = ['description_tfidf_pca.csv', 'position_tfidf_pca.csv', 'requirement_tfidf_pca.csv', 'skills_tfidf_pca.csv', 'title_category_tags_tfidf_pca.csv']
for csv_file in tfidf_pca_file:
    tfidf_pca = pd.read_csv('../data_20220831/'+csv_file).drop(['Unnamed: 0'], axis=1)
    print(csv_file)
    #print(tfidf_pca)
    col_name = csv_file.strip('.csv') # 把skills的s去掉了
    tfidf_pca[col_name] = tfidf_pca.apply(tfidf_pca_merge_fun, axis=1)
    sample = pd.concat([sample, tfidf_pca[[col_name]]], axis=1)

tfidf_pca_col_names = [csv_file.strip('.csv') for csv_file in tfidf_pca_file]
sample[['cv_id', 'jd_id']+tfidf_pca_col_names]

description_tfidf_pca.csv
position_tfidf_pca.csv
requirement_tfidf_pca.csv
skills_tfidf_pca.csv
title_category_tags_tfidf_pca.csv


,cv_id,jd_id,description_tfidf_pca,position_tfidf_pca,requirement_tfidf_pca,kills_tfidf_pca,title_category_tags_tfidf_pca
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,"[-0.2028268203396756, 0.052902330097395, 0.064...","[-0.0393549614722772, -0.1145062990375059, -0....","[-0.0885833919216516, 0.1251912920110739, -0.0...","[0.332299671725948, 0.1661089355047844, 0.1869...","[-0.0428931610734227, -0.1624195901085068, 0.0..."
1,2c9207157bf91042017c0d979cbb047d,2c9207157bcbb76b017bf7f2bbbb3153,"[-0.0930831222487631, 0.0519592769164767, -0.1...","[0.1053468947433308, -0.0266945757513419, -0.0...","[-0.0801264640475711, 0.122717540913069, -0.01...","[0.2361554447739323, 0.1126566470966733, -0.24...","[-0.2347225490786922, 0.0825676460870139, 0.23..."
2,2c9207157bf91042017c7232fbab3be1,2c9207157bf94d99017c108f769405e0,"[-0.077256404167103, -0.195452028548207, -0.22...","[-0.0368041780176452, -0.0857596688737137, -0....","[-0.0644508985020475, 0.1611090469848189, -0.0...","[-0.0922030791609062, -0.1406030646715831, -0....","[-0.1959575316104875, 0.2008839134718935, -0.3..."
3,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,"[-0.0933581483833488, 0.0982068787794115, -0.0...","[0.0490471624206398, -0.0409758387316529, -0.0...","[-0.1339726610662708, -0.1761763831994133, 0.0...","[0.1429396280466366, 0.0217622310718796, -0.14...","[-0.3678224323290787, 0.1854520957699004, 0.32..."
4,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,"[0.0391819610653473, -0.0287343543993758, 0.06...","[-0.0276426492329401, -0.111924304908804, -0.0...","[-0.0136365894358371, -0.0993886855789762, -0....","[-0.1003025481350853, -0.1495209206927971, -0....","[0.0062176220898476, -0.055003735259901, -0.00..."
...,...,...,...,...,...,...,...
83296,8a69f60582c8626e0182c9fc367050e6,8a69c4df82b081d30182c391c1f51bfc,"[-0.1614955741689255, 0.0982571120950793, 0.02...","[-0.0025833865349725, -0.2508193885760124, 0.0...","[-0.1517178304086961, -0.1846200489563693, 0.1...","[0.3241805444120278, -0.0345988526341208, 0.26...","[-0.0954686272882023, -0.0493724436404325, 0.1..."
83297,8a69f60582c8626e0182cedb7ca86dc8,8a69c47b82b07fdc0182b3dcd51b7a71,"[-0.187368032369494, 0.0815596725923325, 0.018...","[0.1067952861660345, -0.225099081085894, -0.06...","[-0.1015761687785662, 0.0302194323476557, 0.04...","[0.0106236588110783, -0.1609745502874504, -0.0...","[-0.1288286036514234, -0.3332390781216969, 0.0..."
83298,8a69f61382b080760182c37a984f3a9a,2c92071580dd3ab70180f18714e119c0,"[0.4185259318945861, -0.0453746812279699, 0.39...","[-0.4597675655720933, 0.5738247473834591, 0.33...","[0.5440054441064249, -0.080866345740466, 0.135...","[-0.2612896440159119, 0.0846500996922437, 0.11...","[0.5515111960481315, 0.4001609626530521, 0.169..."
83299,8a69f64782e59cc90182f37791dc6649,8a69d77e82926b410182b011869b3b74,"[-0.0823327926841566, -0.2134441983861008, -0....","[-0.0126004164756376, -0.1818836719727035, -0....","[-0.0904644960948339, 0.3290683312687502, -0.0...","[0.079180668535461, -0.141120307479467, 0.0457...","[0.0103941678651557, -0.0621128773282237, -0.0..."


# work_duration

In [4]:
# 读取生成的work_duration文件并拼接
work_duration_mean_df = pd.read_csv('../data_20220831/work_duration_mean.csv').drop(['Unnamed: 0'], axis=1)

sample = pd.concat([sample, work_duration_mean_df], axis=1)

columns = ['cv_id', 'jd_id'] + tfidf_pca_col_names + ['work_duration_mean']
sample[columns]

,cv_id,jd_id,description_tfidf_pca,position_tfidf_pca,requirement_tfidf_pca,kills_tfidf_pca,title_category_tags_tfidf_pca,work_duration_mean
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,"[-0.2028268203396756, 0.052902330097395, 0.064...","[-0.0393549614722772, -0.1145062990375059, -0....","[-0.0885833919216516, 0.1251912920110739, -0.0...","[0.332299671725948, 0.1661089355047844, 0.1869...","[-0.0428931610734227, -0.1624195901085068, 0.0...",1209.500000
1,2c9207157bf91042017c0d979cbb047d,2c9207157bcbb76b017bf7f2bbbb3153,"[-0.0930831222487631, 0.0519592769164767, -0.1...","[0.1053468947433308, -0.0266945757513419, -0.0...","[-0.0801264640475711, 0.122717540913069, -0.01...","[0.2361554447739323, 0.1126566470966733, -0.24...","[-0.2347225490786922, 0.0825676460870139, 0.23...",506.000000
2,2c9207157bf91042017c7232fbab3be1,2c9207157bf94d99017c108f769405e0,"[-0.077256404167103, -0.195452028548207, -0.22...","[-0.0368041780176452, -0.0857596688737137, -0....","[-0.0644508985020475, 0.1611090469848189, -0.0...","[-0.0922030791609062, -0.1406030646715831, -0....","[-0.1959575316104875, 0.2008839134718935, -0.3...",811.500000
3,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,"[-0.0933581483833488, 0.0982068787794115, -0.0...","[0.0490471624206398, -0.0409758387316529, -0.0...","[-0.1339726610662708, -0.1761763831994133, 0.0...","[0.1429396280466366, 0.0217622310718796, -0.14...","[-0.3678224323290787, 0.1854520957699004, 0.32...",76.500000
4,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,"[0.0391819610653473, -0.0287343543993758, 0.06...","[-0.0276426492329401, -0.111924304908804, -0.0...","[-0.0136365894358371, -0.0993886855789762, -0....","[-0.1003025481350853, -0.1495209206927971, -0....","[0.0062176220898476, -0.055003735259901, -0.00...",1034.000000
...,...,...,...,...,...,...,...,...
83296,8a69f60582c8626e0182c9fc367050e6,8a69c4df82b081d30182c391c1f51bfc,"[-0.1614955741689255, 0.0982571120950793, 0.02...","[-0.0025833865349725, -0.2508193885760124, 0.0...","[-0.1517178304086961, -0.1846200489563693, 0.1...","[0.3241805444120278, -0.0345988526341208, 0.26...","[-0.0954686272882023, -0.0493724436404325, 0.1...",1268.000000
83297,8a69f60582c8626e0182cedb7ca86dc8,8a69c47b82b07fdc0182b3dcd51b7a71,"[-0.187368032369494, 0.0815596725923325, 0.018...","[0.1067952861660345, -0.225099081085894, -0.06...","[-0.1015761687785662, 0.0302194323476557, 0.04...","[0.0106236588110783, -0.1609745502874504, -0.0...","[-0.1288286036514234, -0.3332390781216969, 0.0...",1583.333333
83298,8a69f61382b080760182c37a984f3a9a,2c92071580dd3ab70180f18714e119c0,"[0.4185259318945861, -0.0453746812279699, 0.39...","[-0.4597675655720933, 0.5738247473834591, 0.33...","[0.5440054441064249, -0.080866345740466, 0.135...","[-0.2612896440159119, 0.0846500996922437, 0.11...","[0.5515111960481315, 0.4001609626530521, 0.169...",487.000000
83299,8a69f64782e59cc90182f37791dc6649,8a69d77e82926b410182b011869b3b74,"[-0.0823327926841566, -0.2134441983861008, -0....","[-0.0126004164756376, -0.1818836719727035, -0....","[-0.0904644960948339, 0.3290683312687502, -0.0...","[0.079180668535461, -0.141120307479467, 0.0457...","[0.0103941678651557, -0.0621128773282237, -0.0...",968.666667


# equal_job

In [5]:
# 读取生成的equal_job文件并拼接
equal_job_df = pd.read_csv('../data_20220831/equal_job.csv').drop(['Unnamed: 0'], axis=1)

sample = pd.concat([sample, equal_job_df], axis=1)

columns = ['cv_id', 'jd_id'] + tfidf_pca_col_names + ['work_duration_mean'] + ['equal_job']
sample[columns]

,cv_id,jd_id,description_tfidf_pca,position_tfidf_pca,requirement_tfidf_pca,kills_tfidf_pca,title_category_tags_tfidf_pca,work_duration_mean,equal_job
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,"[-0.2028268203396756, 0.052902330097395, 0.064...","[-0.0393549614722772, -0.1145062990375059, -0....","[-0.0885833919216516, 0.1251912920110739, -0.0...","[0.332299671725948, 0.1661089355047844, 0.1869...","[-0.0428931610734227, -0.1624195901085068, 0.0...",1209.500000,3
1,2c9207157bf91042017c0d979cbb047d,2c9207157bcbb76b017bf7f2bbbb3153,"[-0.0930831222487631, 0.0519592769164767, -0.1...","[0.1053468947433308, -0.0266945757513419, -0.0...","[-0.0801264640475711, 0.122717540913069, -0.01...","[0.2361554447739323, 0.1126566470966733, -0.24...","[-0.2347225490786922, 0.0825676460870139, 0.23...",506.000000,4
2,2c9207157bf91042017c7232fbab3be1,2c9207157bf94d99017c108f769405e0,"[-0.077256404167103, -0.195452028548207, -0.22...","[-0.0368041780176452, -0.0857596688737137, -0....","[-0.0644508985020475, 0.1611090469848189, -0.0...","[-0.0922030791609062, -0.1406030646715831, -0....","[-0.1959575316104875, 0.2008839134718935, -0.3...",811.500000,0
3,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,"[-0.0933581483833488, 0.0982068787794115, -0.0...","[0.0490471624206398, -0.0409758387316529, -0.0...","[-0.1339726610662708, -0.1761763831994133, 0.0...","[0.1429396280466366, 0.0217622310718796, -0.14...","[-0.3678224323290787, 0.1854520957699004, 0.32...",76.500000,3
4,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,"[0.0391819610653473, -0.0287343543993758, 0.06...","[-0.0276426492329401, -0.111924304908804, -0.0...","[-0.0136365894358371, -0.0993886855789762, -0....","[-0.1003025481350853, -0.1495209206927971, -0....","[0.0062176220898476, -0.055003735259901, -0.00...",1034.000000,1
...,...,...,...,...,...,...,...,...,...
83296,8a69f60582c8626e0182c9fc367050e6,8a69c4df82b081d30182c391c1f51bfc,"[-0.1614955741689255, 0.0982571120950793, 0.02...","[-0.0025833865349725, -0.2508193885760124, 0.0...","[-0.1517178304086961, -0.1846200489563693, 0.1...","[0.3241805444120278, -0.0345988526341208, 0.26...","[-0.0954686272882023, -0.0493724436404325, 0.1...",1268.000000,4
83297,8a69f60582c8626e0182cedb7ca86dc8,8a69c47b82b07fdc0182b3dcd51b7a71,"[-0.187368032369494, 0.0815596725923325, 0.018...","[0.1067952861660345, -0.225099081085894, -0.06...","[-0.1015761687785662, 0.0302194323476557, 0.04...","[0.0106236588110783, -0.1609745502874504, -0.0...","[-0.1288286036514234, -0.3332390781216969, 0.0...",1583.333333,3
83298,8a69f61382b080760182c37a984f3a9a,2c92071580dd3ab70180f18714e119c0,"[0.4185259318945861, -0.0453746812279699, 0.39...","[-0.4597675655720933, 0.5738247473834591, 0.33...","[0.5440054441064249, -0.080866345740466, 0.135...","[-0.2612896440159119, 0.0846500996922437, 0.11...","[0.5515111960481315, 0.4001609626530521, 0.169...",487.000000,5
83299,8a69f64782e59cc90182f37791dc6649,8a69d77e82926b410182b011869b3b74,"[-0.0823327926841566, -0.2134441983861008, -0....","[-0.0126004164756376, -0.1818836719727035, -0....","[-0.0904644960948339, 0.3290683312687502, -0.0...","[0.079180668535461, -0.141120307479467, 0.0457...","[0.0103941678651557, -0.0621128773282237, -0.0...",968.666667,1


# salary

In [6]:
# 读取生成的salary文件并拼接
salary_df = pd.read_csv('../data_20220831/salary.csv').drop(['Unnamed: 0'], axis=1)

sample = pd.concat([sample, salary_df], axis=1)

columns = ['cv_id', 'jd_id'] + tfidf_pca_col_names + ['work_duration_mean'] + ['equal_job'] + ['parsed_desiredSalary', 'parsed_currentSalary']
sample[columns]

,cv_id,jd_id,description_tfidf_pca,position_tfidf_pca,requirement_tfidf_pca,kills_tfidf_pca,title_category_tags_tfidf_pca,work_duration_mean,equal_job,parsed_desiredSalary,parsed_currentSalary
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,"[-0.2028268203396756, 0.052902330097395, 0.064...","[-0.0393549614722772, -0.1145062990375059, -0....","[-0.0885833919216516, 0.1251912920110739, -0.0...","[0.332299671725948, 0.1661089355047844, 0.1869...","[-0.0428931610734227, -0.1624195901085068, 0.0...",1209.500000,3,-1,-1
1,2c9207157bf91042017c0d979cbb047d,2c9207157bcbb76b017bf7f2bbbb3153,"[-0.0930831222487631, 0.0519592769164767, -0.1...","[0.1053468947433308, -0.0266945757513419, -0.0...","[-0.0801264640475711, 0.122717540913069, -0.01...","[0.2361554447739323, 0.1126566470966733, -0.24...","[-0.2347225490786922, 0.0825676460870139, 0.23...",506.000000,4,-1,-1
2,2c9207157bf91042017c7232fbab3be1,2c9207157bf94d99017c108f769405e0,"[-0.077256404167103, -0.195452028548207, -0.22...","[-0.0368041780176452, -0.0857596688737137, -0....","[-0.0644508985020475, 0.1611090469848189, -0.0...","[-0.0922030791609062, -0.1406030646715831, -0....","[-0.1959575316104875, 0.2008839134718935, -0.3...",811.500000,0,-1,-1
3,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,"[-0.0933581483833488, 0.0982068787794115, -0.0...","[0.0490471624206398, -0.0409758387316529, -0.0...","[-0.1339726610662708, -0.1761763831994133, 0.0...","[0.1429396280466366, 0.0217622310718796, -0.14...","[-0.3678224323290787, 0.1854520957699004, 0.32...",76.500000,3,-1,-1
4,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,"[0.0391819610653473, -0.0287343543993758, 0.06...","[-0.0276426492329401, -0.111924304908804, -0.0...","[-0.0136365894358371, -0.0993886855789762, -0....","[-0.1003025481350853, -0.1495209206927971, -0....","[0.0062176220898476, -0.055003735259901, -0.00...",1034.000000,1,450,-1
...,...,...,...,...,...,...,...,...,...,...,...
83296,8a69f60582c8626e0182c9fc367050e6,8a69c4df82b081d30182c391c1f51bfc,"[-0.1614955741689255, 0.0982571120950793, 0.02...","[-0.0025833865349725, -0.2508193885760124, 0.0...","[-0.1517178304086961, -0.1846200489563693, 0.1...","[0.3241805444120278, -0.0345988526341208, 0.26...","[-0.0954686272882023, -0.0493724436404325, 0.1...",1268.000000,4,-1,-1
83297,8a69f60582c8626e0182cedb7ca86dc8,8a69c47b82b07fdc0182b3dcd51b7a71,"[-0.187368032369494, 0.0815596725923325, 0.018...","[0.1067952861660345, -0.225099081085894, -0.06...","[-0.1015761687785662, 0.0302194323476557, 0.04...","[0.0106236588110783, -0.1609745502874504, -0.0...","[-0.1288286036514234, -0.3332390781216969, 0.0...",1583.333333,3,120,120
83298,8a69f61382b080760182c37a984f3a9a,2c92071580dd3ab70180f18714e119c0,"[0.4185259318945861, -0.0453746812279699, 0.39...","[-0.4597675655720933, 0.5738247473834591, 0.33...","[0.5440054441064249, -0.080866345740466, 0.135...","[-0.2612896440159119, 0.0846500996922437, 0.11...","[0.5515111960481315, 0.4001609626530521, 0.169...",487.000000,5,300,216
83299,8a69f64782e59cc90182f37791dc6649,8a69d77e82926b410182b011869b3b74,"[-0.0823327926841566, -0.2134441983861008, -0....","[-0.0126004164756376, -0.1818836719727035, -0....","[-0.0904644960948339, 0.3290683312687502, -0.0...","[0.079180668535461, -0.141120307479467, 0.0457...","[0.0103941678651557, -0.0621128773282237, -0.0...",968.666667,1,900,900


In [ ]:
# 将new特征保存
sample[columns].to_csv('../data_20220831/sample_new_feature_20220831.csv')

In [ ]:
'''
特别注意，以上的拼接都是基于raw_cvjd_20220831_spark.csv数据

description_tfidf_pca
position_tfidf_pca
requirement_tfidf_pca
kills_tfidf_pca
title_category_tags_tfidf_pca
work_duration_mean
equal_job
parsed_desiredSalary
parsed_currentSalary
这些列没有cv_id和jd_id，仅仅是通过行拼接生成数据集（应该是没有行数据错位的情况）。
'''

# 原始特征

In [7]:
# 这些特征在原始的cv和jd表里
raw_columns = ['age', 'recruit_number', 'workYear', 'min_month_salary', 'max_month_salary', 'pay_months']


In [ ]:
# 将raw特征保存
sample[['cv_id', 'jd_id'] + raw_columns].to_csv('../data_20220831/sample_raw_feature_20220831.csv')

# ext特征

In [8]:
# 这些特征在sample pipeline的表里
ext_columns = ['school_labels_cv', 'school_labels_jd', 'expectlocation_range_cv', 'location_range_jd', 'degree_index_cv', 'degree_index_jd', 'workyear_range_jd', 'gender_index_cv', 'job_intentstatus_cv']


In [9]:
# 读取包含label的sample表
all_sample = pd.read_csv('../data_20220831/sample_20220831_spark.csv').drop('Unnamed: 0', axis=1)

/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_74508/3021191761.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  all_sample = pd.read_csv('../data_20220831/sample_20220831_spark.csv').drop('Unnamed: 0', axis=1)


In [ ]:
# 将ext特征保存
all_sample[['cv_id', 'jd_id'] + ext_columns].to_csv('../data_20220831/sample_ext_feature_20220831.csv')

# 拼接数据集

In [10]:
# 初步怀疑这里的feature的cvid和jdid有重复
base_path = '../data_20220831/'
raw_feature_file = 'sample_raw_feature_20220831.csv'
ext_feature_file = 'sample_ext_feature_20220831.csv'
new_feature_file = 'sample_new_feature_20220831.csv'

# 读取特征文件并以id去重
raw_feature = pd.read_csv(base_path + raw_feature_file).drop('Unnamed: 0', axis=1).drop_duplicates(subset=['cv_id', 'jd_id'], keep='first')
ext_feature = pd.read_csv(base_path + ext_feature_file).drop('Unnamed: 0', axis=1).drop_duplicates(subset=['cv_id', 'jd_id'], keep='first')
new_feature = pd.read_csv(base_path + new_feature_file).drop('Unnamed: 0', axis=1).drop_duplicates(subset=['cv_id', 'jd_id'], keep='first')

In [11]:
# 3份特征文件拼接
medium = raw_feature.join(ext_feature.set_index(['cv_id', 'jd_id']), on=['cv_id', 'jd_id'], how='inner')
raw_ext_new_feature = medium.join(new_feature.set_index(['cv_id', 'jd_id']), on=['cv_id', 'jd_id'], how='inner')
raw_ext_new_feature

,cv_id,jd_id,age,recruit_number,workYear,min_month_salary,max_month_salary,pay_months,school_labels_cv,school_labels_jd,...,job_intentstatus_cv,description_tfidf_pca,position_tfidf_pca,requirement_tfidf_pca,kills_tfidf_pca,title_category_tags_tfidf_pca,work_duration_mean,equal_job,parsed_desiredSalary,parsed_currentSalary
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,36.0,1,13.0,NaN,NaN,NaN,[],[],...,3,"[-0.2028268203396756, 0.052902330097395, 0.064...","[-0.0393549614722772, -0.1145062990375059, -0....","[-0.0885833919216516, 0.1251912920110739, -0.0...","[0.332299671725948, 0.1661089355047844, 0.1869...","[-0.0428931610734227, -0.1624195901085068, 0.0...",1209.500000,3,-1,-1
1,2c9207157bf91042017c0d979cbb047d,2c9207157bcbb76b017bf7f2bbbb3153,29.0,1,7.0,NaN,NaN,NaN,[],[],...,2,"[-0.0930831222487631, 0.0519592769164767, -0.1...","[0.1053468947433308, -0.0266945757513419, -0.0...","[-0.0801264640475711, 0.122717540913069, -0.01...","[0.2361554447739323, 0.1126566470966733, -0.24...","[-0.2347225490786922, 0.0825676460870139, 0.23...",506.000000,4,-1,-1
2,2c9207157bf91042017c7232fbab3be1,2c9207157bf94d99017c108f769405e0,NaN,1,14.0,NaN,NaN,NaN,[],[],...,3,"[-0.077256404167103, -0.195452028548207, -0.22...","[-0.0368041780176452, -0.0857596688737137, -0....","[-0.0644508985020475, 0.1611090469848189, -0.0...","[-0.0922030791609062, -0.1406030646715831, -0....","[-0.1959575316104875, 0.2008839134718935, -0.3...",811.500000,0,-1,-1
3,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,27.0,1,3.0,NaN,NaN,NaN,"['211', '双一流']",[],...,3,"[-0.0933581483833488, 0.0982068787794115, -0.0...","[0.0490471624206398, -0.0409758387316529, -0.0...","[-0.1339726610662708, -0.1761763831994133, 0.0...","[0.1429396280466366, 0.0217622310718796, -0.14...","[-0.3678224323290787, 0.1854520957699004, 0.32...",76.500000,3,-1,-1
4,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,28.0,1,5.0,25.0,45.0,12.0,"['211', '双一流']",[],...,3,"[0.0391819610653473, -0.0287343543993758, 0.06...","[-0.0276426492329401, -0.111924304908804, -0.0...","[-0.0136365894358371, -0.0993886855789762, -0....","[-0.1003025481350853, -0.1495209206927971, -0....","[0.0062176220898476, -0.055003735259901, -0.00...",1034.000000,1,450,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83296,8a69f60582c8626e0182c9fc367050e6,8a69c4df82b081d30182c391c1f51bfc,31.0,1,11.0,30.0,40.0,12.0,[],[],...,0,"[-0.1614955741689255, 0.0982571120950793, 0.02...","[-0.0025833865349725, -0.2508193885760124, 0.0...","[-0.1517178304086961, -0.1846200489563693, 0.1...","[0.3241805444120278, -0.0345988526341208, 0.26...","[-0.0954686272882023, -0.0493724436404325, 0.1...",1268.000000,4,-1,-1
83297,8a69f60582c8626e0182cedb7ca86dc8,8a69c47b82b07fdc0182b3dcd51b7a71,36.0,5,14.0,15.0,30.0,12.0,[],[],...,3,"[-0.187368032369494, 0.0815596725923325, 0.018...","[0.1067952861660345, -0.225099081085894, -0.06...","[-0.1015761687785662, 0.0302194323476557, 0.04...","[0.0106236588110783, -0.1609745502874504, -0.0...","[-0.1288286036514234, -0.3332390781216969, 0.0...",1583.333333,3,120,120
83298,8a69f61382b080760182c37a984f3a9a,2c92071580dd3ab70180f18714e119c0,27.0,30,5.0,15.0,30.0,14.0,[],[],...,0,"[0.4185259318945861, -0.0453746812279699, 0.39...","[-0.4597675655720933, 0.5738247473834591, 0.33...","[0.5440054441064249, -0.080866345740466, 0.135...","[-0.2612896440159119, 0.0846500996922437, 0.11...","[0.5515111960481315, 0.4001609626530521, 0.169...",487.000000,5,300,216
83299,8a69f64782e59cc90182f37791dc6649,8a69d77e82926b410182b011869b3b74,43.0,1,16.0,30.0,50.0,13.0,[],[],...,3,"[-0.0823327926841566, -0.2134441983861008, -0....","[-0.0126004164756376, -0.1818836719727035, -0....","[-0.0904644960948339, 0.3290683312687502, -0.0...","[0.079180668535461, -0.141120307479467, 0.0457...","[0.0103941678651557, -0.0621128773282237, -0.0...",968.666667,1,900,900


In [12]:
# 读取带label的sample文件并以id去重
all_sample = pd.read_csv('../data_20220831/sample_20220831_spark.csv').drop('Unnamed: 0', axis=1).drop_duplicates(subset=['cv_id', 'jd_id'], keep='first')
all_sample

/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_74508/220314284.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  all_sample = pd.read_csv('../data_20220831/sample_20220831_spark.csv').drop('Unnamed: 0', axis=1).drop_duplicates(subset=['cv_id', 'jd_id'], keep='first')


,cv_id,jd_id,cv_code,jd_code,last_opt,da_label,rank_label,title,deadline,category_id,...,industry_cut,industry_jd2vec,industry_vec2,majorName_cut,majorName_jd2vec,majorName_vec2,skills_cut,skills_jd2vec,skills_vec2,ds
0,2c9207157bcb9549017bce3ea49b00fa,2c9207157bcbb76b017bce3d99ba059a,C972256920524165184,JD20210910134407574550,drop,1,1,市场策划总监,2021-11-30 23:59:59.0,140404,...,"['#', 'null']","[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....","['舞蹈', '学']","[0.2290763258934021, -0.14894524216651917, 0.0...","[-0.033116504549980164, -0.058041930198669434,...","['[', '""', 'Ai', '""', ',', '""', 'Ar', '""', ','...","[0.37405160069465637, 0.08108291774988174, -0....","[0.18812726438045502, -0.014001657254993916, -...",20220831
1,2c9207157bf91042017c1040330d04d3,2c9207157bcbb76b017bec76efca1ca5,C972256920532553817,JD20210916103438584949,link,-1,1,解决方案总监/经理,2021-10-31 23:59:59.0,101201,...,"['通信', '技术', '提供商']","[0.18720732629299164, -0.24131253361701965, -0...","[0.768592119216919, -0.16999222338199615, -0.4...",['计算机技术'],"[0.06550661474466324, -0.30920350551605225, -0...","[0.00835432019084692, -0.6533162593841553, -0....","['[', '""', 'it', '规划', '咨询', '""', ',', '""', '云...","[0.32306215167045593, 0.09080911427736282, -0....","[0.5299833416938782, -0.1909421980381012, -0.2...",20220831
2,2c9207157bf91042017c10f871d309f4,2c9207157f347800017f624770866105,C972256920545136700,JD20220216104801688663,drop,1,1,医学博士投资人,NaN,180101,...,"['基金', '/', '证券', '/', '期货', '/', '投资']","[0.022260336205363274, -0.03374471887946129, -...","[0.30257120728492737, -0.31254786252975464, -0...","['生物医学', '工程']","[0.01792692393064499, -0.1300506740808487, -0....","[0.005150213837623596, 0.15490269660949707, -0...","['[', '""', 'VC', '""', ',', '""', 'PPT', '""', ',...","[0.3084435760974884, 0.07542910426855087, -0.2...","[0.09389838576316833, 0.04999205097556114, -0....",20220831
3,2c9207157bf91042017c62fce2583411,2c9207157b9b9199017bca11fd3d3f11,C972256920557719589,JD20210908152800055103,reject,2,2,高级用户运营专家,2021-12-31 23:59:59.0,130405,...,"['#', 'null']","[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....",['市场营销'],"[0.6256594657897949, -0.07856443524360657, 0.0...","[0.7786060571670532, -0.8168095946311951, -0.6...","['[', '""', 'Vi', '""', ',', '""', 'Pr', '""', ','...","[0.3983086943626404, 0.11806842684745789, -0.2...","[0.29188045859336853, -0.04354493319988251, -0...",20220831
4,2c9207157bf91042017c6ecc163d3aa8,2c9207157bf94d99017c6ecb294930b1,C972256920557719638,JD20211011173942648108,accept,3,3,咨询总监,2021-10-31 23:59:59.0,260401,...,"['#', 'null']","[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....",['金融学'],"[0.11331989616155624, -0.1490677297115326, -0....","[-0.3050949275493622, -0.4833742380142212, -2....","['[', '""', 'PPT', '""', ',', '""', 'Word', '""', ...","[0.4078637957572937, 0.07882367819547653, -0.2...","[0.1595805436372757, -0.0552152618765831, -0.3...",20220831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83296,8a69d7c28219945301823a12e0266463,8a69c4df82b081d30182c9ec1ecd1c67,C1001558237016887296,JD20220823165719130900,link,-1,1,声学工程师,NaN,100801,...,"['#', 'null']","[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....","['韩国', '永进', '大学']","[-0.016853807494044304, -0.16189433634281158, ...","[0.4639318585395813, 0.4096287488937378, 0.103...","['CAD', ',', 'Proe', ',', 'Ansys', ',', '优化', ...","[0.29724088311195374, -0.06454341858625412, -0...","[-0.07170484960079193, 0.2823019027709961, -0....",20220831
83297,8a69d7c282199453018246ebf6b60643,8a69f67f82198d89018234adfcd91510,C1002462334712680448,JD20220725172633862905,post,-1,2,销售代表,NaN,140301,...,"['#', 'null']","[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....","['河南', '科技', '大学']","[0.4

In [ ]:
# 拼接特征和label并保存
label_df = all_sample[['cv_id', 'jd_id', 'da_label', 'rank_label']]
raw_ext_new_feature_label = raw_ext_new_feature.join(label_df.set_index(['cv_id', 'jd_id']), on=['cv_id', 'jd_id'], how='inner')
raw_ext_new_feature_label.columns
raw_ext_new_feature_label.to_csv('../data_20220831/raw_ext_new_feature_label.csv')

# 复推数据标签

In [13]:
#audit_state（4未处理：过滤，5放弃不会有order，6同意与101对应）
#resume_type: 2-AI人岗, 5-资深人工撮合, 6-复推, 普通猎头单不在其中（普通猎头正样本要去order里找）
#compatibility 规则打分(0-100)，AI人岗类型(resume_type=2)有效
#resume_source：忽略，resume_type=2，5，6时全都等于2，代表AI人岗和复推

# 读取带复推标签的数据并去重
resume_type_df = pd.read_csv('../data_20220831/sample_resume_type_20220831_spark.csv').drop_duplicates(subset=['cv_id', 'jd_id'], keep='first').drop('Unnamed: 0', axis=1)

resume_type_df

,cv_id,jd_id,resume_type
0,2c9207157bcb9549017bce3ea49b00fa,2c9207157bcbb76b017bce3d99ba059a,NaN
1,2c9207157bf91042017c1040330d04d3,2c9207157bcbb76b017bec76efca1ca5,4.0
2,2c9207157bf91042017c10f871d309f4,2c9207157f347800017f624770866105,4.0
3,2c9207157bf91042017c62fce2583411,2c9207157b9b9199017bca11fd3d3f11,NaN
4,2c9207157bf91042017c6ecc163d3aa8,2c9207157bf94d99017c6ecb294930b1,NaN
...,...,...,...
85926,8a69d7c28219945301823a12e0266463,8a69c4df82b081d30182c9ec1ecd1c67,5.0
85927,8a69d7c282199453018246ebf6b60643,8a69f67f82198d89018234adfcd91510,4.0
85928,8a69d7c282199453018247bb00cb68a3,8a69c468826ee9b40182831bedfc5e79,4.0
85929,8a69d7c282199453018247bb00cb68a3,8a69f6f8823a93b9018247c30aca4e3d,4.0


In [ ]:
# 拼接特征、label和复推标签并保存
raw_ext_new_feature_label_resume_type = raw_ext_new_feature_label.join(resume_type_df.set_index(['cv_id', 'jd_id']), on=['cv_id', 'jd_id'], how='inner')
raw_ext_new_feature_label_resume_type.to_csv('../data_20220831/raw_ext_new_feature_label_resume_type.csv')

# 复推集剔除与训练集分层切分

In [ ]:
import copy

# 不必理会这里的命名
test_df = copy.deepcopy(raw_ext_new_feature_label_resume_type)


In [ ]:
# 去除复推数据
train_test_data = test_df[~(test_df['resume_type']==6)]

In [ ]:
# rank_label里有-1标签的数据，去除da_label=-1&rank_label=0（未收录）以及da_label=-1&rank_label=1的数据
train_test_data_filter = train_test_data[~((train_test_data['da_label']==-1)&(train_test_data['rank_label']==1))]
train_test_data_filter['rank_label'].value_counts()

# 和文档中的数据规模保持一致
# https://jinrirencai.feishu.cn/docx/doxcnsFcNXW3p3PlgZ471mAGP7t

In [ ]:
# 分层
train_test_data_filter_0 = train_test_data_filter[train_test_data_filter['rank_label']==0]
train_test_data_filter_1 = train_test_data_filter[train_test_data_filter['rank_label']==1]
train_test_data_filter_2 = train_test_data_filter[train_test_data_filter['rank_label']==2]
train_test_data_filter_3 = train_test_data_filter[train_test_data_filter['rank_label']==3]


In [ ]:
# 分层采样
from sklearn.model_selection import train_test_split
loop_list = [train_test_data_filter_0, train_test_data_filter_1, train_test_data_filter_2, train_test_data_filter_3]
sample_train = None
sample_test = None
for data in loop_list:
    train, test = train_test_split(data, test_size=0.2, random_state=10)
    sample_train = pd.concat([sample_train, train])
    sample_test = pd.concat([sample_test, test])


In [ ]:
print(len(sample_train))
print(len(sample_test))

In [ ]:
sample_train['rank_label'].value_counts()
sample_test['rank_label'].value_counts()

In [ ]:
sample_train.to_csv('../data_20220831/sample_train.csv')
sample_test.to_csv('../data_20220831/sample_test.csv')

# sample_train缺失值填充

In [15]:
import pandas as pd
sample_train = pd.read_csv('../data_20220831/sample_train.csv').drop('Unnamed: 0', axis=1)

In [16]:
sample_train

,cv_id,jd_id,age,recruit_number,workYear,min_month_salary,max_month_salary,pay_months,school_labels_cv,school_labels_jd,...,requirement_tfidf_pca,kills_tfidf_pca,title_category_tags_tfidf_pca,work_duration_mean,equal_job,parsed_desiredSalary,parsed_currentSalary,da_label,rank_label,resume_type
0,2c9207158059e82b018067f52ee520f1,2c9207157ef60164017ef604e195022e,26.0,1,4.0,25.0,32.0,16.0,"['211', '双一流']",[],...,"[0.0146099317606446, -0.0973321531897229, -0.0...","[0.0267966354783416, 0.1949045208271133, -0.15...","[-0.0268220597366096, 0.0146753642354594, -0.1...",270.00,3,180,-1,0,0,2.0
1,2c920715804c9ef601805374e101570a,2c92071580b3a1010180bda0e1166bfd,46.0,1,23.0,80.0,100.0,12.0,"['985', '211', '双一流']",[],...,"[-0.0804330166663323, -0.265194138646796, 0.05...","[0.1907121220593182, 0.2593476022337883, 0.109...","[-0.1403481086160699, 0.101545233205827, -0.14...",1019.50,5,-1,-1,0,0,2.0
2,2c9207157ef7ed7f017f075fc6640e79,2c9207157d2f0ec5017d5a854c143177,36.0,1,12.0,NaN,NaN,NaN,[],['双一流'],...,"[-0.0967045533557524, -0.0158876913753405, -0....","[0.1350832166503662, 0.0570712078989811, -0.16...","[0.0041959442715183, -0.0689690014332659, -0.0...",832.00,3,-1,540,0,0,2.0
3,8a69c4e9824fd55f018258b725082f07,8a69f68e82198e9101822545359e0a0d,28.0,1,5.0,40.0,70.0,12.0,[],[],...,"[0.1213571418999401, 0.0752965052240779, -0.04...","[-0.0822782506729067, 0.0309579678495507, -0.0...","[0.0593428963865967, -0.0521583781812485, -0.0...",401.40,4,-1,288,0,0,5.0
4,2c920715800e8898018022a768d41cc3,2c9207157f347800017f72dd6e04248b,33.0,1,8.0,20.0,30.0,16.0,['双一流'],[],...,"[-0.0555047818278877, -0.0795936243595016, 0.0...","[-0.0508683560010078, 0.2361558331204752, -0.1...","[-0.4085584894502868, 0.2043626298020381, 0.02...",692.00,2,-1,-1,0,0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58499,2c9207157e58b769017e703d68483561,2c9207157dc878e5017ddab2b223521b,NaN,15,NaN,8.0,15.0,12.0,[],[],...,"[-0.0668460252602592, 0.0700903434401426, 0.05...","[-0.0950710757884233, -0.1473097543095731, -0....","[0.0196958641886768, -0.0692635968080485, -0.0...",NaN,0,-1,-1,3,3,4.0
58500,8a69c4e88262c85901826bc2582c33d0,8a69f61482627e7a0182628269db2b56,32.0,6,9.0,11.0,28.0,14.0,[],[],...,"[-0.0098772535847565, 0.1301945044403502, -0.0...","[-0.1535405985665613, -0.0679644279924362, -0....","[0.1167680749390213, -0.0366297624844817, -0.0...",814.50,4,270,240,3,3,4.0
58501,2c9207157ef7ed7f017f0a990d4d23a0,2c9207157ef7ed16017f0c28fa6c5e48,42.0,1,13.0,40.0,90.0,12.0,"['211', '双一流']",[],...,"[-0.097456235834622, -0.0709741819063322, 0.27...","[0.2580047261706898, -0.1447766777098391, 0.28...","[-0.0871293740917814, -0.0944614800646356, 0.1...",1141.75,4,-1,-1,3,3,4.0
58502,8a69d7a68219933d01821fc0c69843ea,2c9207158114fb2901815c03bb257372,30.0,1,6.0,20.0,35.0,14.0,[],[],...,"[0.1227189818078986, 0.0444096728787962, -0.01...","[-0.2917169624226904, 0.0940038684861844, 0.06...","[0.1580939681081193, 0.0353872320416606, -0.09...",501.50,4,600,247,3,3,4.0


In [17]:
# 中位数填充
from sklearn.impute import SimpleImputer

imp = SimpleImputer(missing_values=np.nan, strategy='median')
sample_train['age'] = imp.fit_transform(sample_train[['age']])
sample_train['workYear'] = imp.fit_transform(sample_train[['workYear']])
sample_train['min_month_salary'] = imp.fit_transform(sample_train[['min_month_salary']])
sample_train['max_month_salary'] = imp.fit_transform(sample_train[['max_month_salary']])
sample_train['pay_months'] = imp.fit_transform(sample_train[['pay_months']])
sample_train['work_duration_mean'] = imp.fit_transform(sample_train[['work_duration_mean']])

imp = SimpleImputer(missing_values=-1, strategy='median')
# 这两个只能选一个接入模型
sample_train['parsed_desiredSalary'] = imp.fit_transform(sample_train[['parsed_desiredSalary']])
sample_train['parsed_currentSalary'] = imp.fit_transform(sample_train[['parsed_currentSalary']])
sample_train


,cv_id,jd_id,age,recruit_number,workYear,min_month_salary,max_month_salary,pay_months,school_labels_cv,school_labels_jd,...,requirement_tfidf_pca,kills_tfidf_pca,title_category_tags_tfidf_pca,work_duration_mean,equal_job,parsed_desiredSalary,parsed_currentSalary,da_label,rank_label,resume_type
0,2c9207158059e82b018067f52ee520f1,2c9207157ef60164017ef604e195022e,26.0,1,4.0,25.0,32.0,16.0,"['211', '双一流']",[],...,"[0.0146099317606446, -0.0973321531897229, -0.0...","[0.0267966354783416, 0.1949045208271133, -0.15...","[-0.0268220597366096, 0.0146753642354594, -0.1...",270.00,3,180.0,300.0,0,0,2.0
1,2c920715804c9ef601805374e101570a,2c92071580b3a1010180bda0e1166bfd,46.0,1,23.0,80.0,100.0,12.0,"['985', '211', '双一流']",[],...,"[-0.0804330166663323, -0.265194138646796, 0.05...","[0.1907121220593182, 0.2593476022337883, 0.109...","[-0.1403481086160699, 0.101545233205827, -0.14...",1019.50,5,360.0,300.0,0,0,2.0
2,2c9207157ef7ed7f017f075fc6640e79,2c9207157d2f0ec5017d5a854c143177,36.0,1,12.0,25.0,40.0,14.0,[],['双一流'],...,"[-0.0967045533557524, -0.0158876913753405, -0....","[0.1350832166503662, 0.0570712078989811, -0.16...","[0.0041959442715183, -0.0689690014332659, -0.0...",832.00,3,360.0,540.0,0,0,2.0
3,8a69c4e9824fd55f018258b725082f07,8a69f68e82198e9101822545359e0a0d,28.0,1,5.0,40.0,70.0,12.0,[],[],...,"[0.1213571418999401, 0.0752965052240779, -0.04...","[-0.0822782506729067, 0.0309579678495507, -0.0...","[0.0593428963865967, -0.0521583781812485, -0.0...",401.40,4,360.0,288.0,0,0,5.0
4,2c920715800e8898018022a768d41cc3,2c9207157f347800017f72dd6e04248b,33.0,1,8.0,20.0,30.0,16.0,['双一流'],[],...,"[-0.0555047818278877, -0.0795936243595016, 0.0...","[-0.0508683560010078, 0.2361558331204752, -0.1...","[-0.4085584894502868, 0.2043626298020381, 0.02...",692.00,2,360.0,300.0,0,0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58499,2c9207157e58b769017e703d68483561,2c9207157dc878e5017ddab2b223521b,32.0,15,8.0,8.0,15.0,12.0,[],[],...,"[-0.0668460252602592, 0.0700903434401426, 0.05...","[-0.0950710757884233, -0.1473097543095731, -0....","[0.0196958641886768, -0.0692635968080485, -0.0...",783.50,0,360.0,300.0,3,3,4.0
58500,8a69c4e88262c85901826bc2582c33d0,8a69f61482627e7a0182628269db2b56,32.0,6,9.0,11.0,28.0,14.0,[],[],...,"[-0.0098772535847565, 0.1301945044403502, -0.0...","[-0.1535405985665613, -0.0679644279924362, -0....","[0.1167680749390213, -0.0366297624844817, -0.0...",814.50,4,270.0,240.0,3,3,4.0
58501,2c9207157ef7ed7f017f0a990d4d23a0,2c9207157ef7ed16017f0c28fa6c5e48,42.0,1,13.0,40.0,90.0,12.0,"['211', '双一流']",[],...,"[-0.097456235834622, -0.0709741819063322, 0.27...","[0.2580047261706898, -0.1447766777098391, 0.28...","[-0.0871293740917814, -0.0944614800646356, 0.1...",1141.75,4,360.0,300.0,3,3,4.0
58502,8a69d7a68219933d01821fc0c69843ea,2c9207158114fb2901815c03bb257372,30.0,1,6.0,20.0,35.0,14.0,[],[],...,"[0.1227189818078986, 0.0444096728787962, -0.01...","[-0.2917169624226904, 0.0940038684861844, 0.06...","[0.1580939681081193, 0.0353872320416606, -0.09...",501.50,4,600.0,247.0,3,3,4.0


# sample_train手动cross

In [18]:
# 首先把school_labels_cv和school_labels_jd转成列表
def school_lables_to_list(series, school_col):
    school_labels = series[school_col]
    school_labels_list = [item.strip('\'') for item in school_labels.strip('[]').replace(' ', '').split(',')]
    if school_labels_list[0] == '':
        return []
    return school_labels_list
sample_train['school_labels_cv_list'] = sample_train.apply(school_lables_to_list, axis=1, args=('school_labels_cv', ))
sample_train['school_labels_jd_list'] = sample_train.apply(school_lables_to_list, axis=1, args=('school_labels_jd', ))

sample_train

,cv_id,jd_id,age,recruit_number,workYear,min_month_salary,max_month_salary,pay_months,school_labels_cv,school_labels_jd,...,title_category_tags_tfidf_pca,work_duration_mean,equal_job,parsed_desiredSalary,parsed_currentSalary,da_label,rank_label,resume_type,school_labels_cv_list,school_labels_jd_list
0,2c9207158059e82b018067f52ee520f1,2c9207157ef60164017ef604e195022e,26.0,1,4.0,25.0,32.0,16.0,"['211', '双一流']",[],...,"[-0.0268220597366096, 0.0146753642354594, -0.1...",270.00,3,180.0,300.0,0,0,2.0,"[211, 双一流]",[]
1,2c920715804c9ef601805374e101570a,2c92071580b3a1010180bda0e1166bfd,46.0,1,23.0,80.0,100.0,12.0,"['985', '211', '双一流']",[],...,"[-0.1403481086160699, 0.101545233205827, -0.14...",1019.50,5,360.0,300.0,0,0,2.0,"[985, 211, 双一流]",[]
2,2c9207157ef7ed7f017f075fc6640e79,2c9207157d2f0ec5017d5a854c143177,36.0,1,12.0,25.0,40.0,14.0,[],['双一流'],...,"[0.0041959442715183, -0.0689690014332659, -0.0...",832.00,3,360.0,540.0,0,0,2.0,[],[双一流]
3,8a69c4e9824fd55f018258b725082f07,8a69f68e82198e9101822545359e0a0d,28.0,1,5.0,40.0,70.0,12.0,[],[],...,"[0.0593428963865967, -0.0521583781812485, -0.0...",401.40,4,360.0,288.0,0,0,5.0,[],[]
4,2c920715800e8898018022a768d41cc3,2c9207157f347800017f72dd6e04248b,33.0,1,8.0,20.0,30.0,16.0,['双一流'],[],...,"[-0.4085584894502868, 0.2043626298020381, 0.02...",692.00,2,360.0,300.0,0,0,2.0,[双一流],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58499,2c9207157e58b769017e703d68483561,2c9207157dc878e5017ddab2b223521b,32.0,15,8.0,8.0,15.0,12.0,[],[],...,"[0.0196958641886768, -0.0692635968080485, -0.0...",783.50,0,360.0,300.0,3,3,4.0,[],[]
58500,8a69c4e88262c85901826bc2582c33d0,8a69f61482627e7a0182628269db2b56,32.0,6,9.0,11.0,28.0,14.0,[],[],...,"[0.1167680749390213, -0.0366297624844817, -0.0...",814.50,4,270.0,240.0,3,3,4.0,[],[]
58501,2c9207157ef7ed7f017f0a990d4d23a0,2c9207157ef7ed16017f0c28fa6c5e48,42.0,1,13.0,40.0,90.0,12.0,"['211', '双一流']",[],...,"[-0.0871293740917814, -0.0944614800646356, 0.1...",1141.75,4,360.0,300.0,3,3,4.0,"[211, 双一流]",[]
58502,8a69d7a68219933d01821fc0c69843ea,2c9207158114fb2901815c03bb257372,30.0,1,6.0,20.0,35.0,14.0,[],[],...,"[0.1580939681081193, 0.0353872320416606, -0.09...",501.50,4,600.0,247.0,3,3,4.0,[],[]


In [19]:
# school等级交叉
def school_intersect_cvjd(series, school_labels_cv, school_labels_jd):
    school_labels_cv = series[school_labels_cv]
    school_labels_jd = series[school_labels_jd]
    # jd没有学历要求or解析不出来
    if not school_labels_jd:
        return 1
    if set(school_labels_cv).intersection(set(school_labels_jd)):
        return 1
    return 0

sample_train['school_intersect_cvjd'] = sample_train.apply(school_intersect_cvjd, axis=1, args=('school_labels_cv_list', 'school_labels_jd_list'))
sample_train

,cv_id,jd_id,age,recruit_number,workYear,min_month_salary,max_month_salary,pay_months,school_labels_cv,school_labels_jd,...,work_duration_mean,equal_job,parsed_desiredSalary,parsed_currentSalary,da_label,rank_label,resume_type,school_labels_cv_list,school_labels_jd_list,school_intersect_cvjd
0,2c9207158059e82b018067f52ee520f1,2c9207157ef60164017ef604e195022e,26.0,1,4.0,25.0,32.0,16.0,"['211', '双一流']",[],...,270.00,3,180.0,300.0,0,0,2.0,"[211, 双一流]",[],1
1,2c920715804c9ef601805374e101570a,2c92071580b3a1010180bda0e1166bfd,46.0,1,23.0,80.0,100.0,12.0,"['985', '211', '双一流']",[],...,1019.50,5,360.0,300.0,0,0,2.0,"[985, 211, 双一流]",[],1
2,2c9207157ef7ed7f017f075fc6640e79,2c9207157d2f0ec5017d5a854c143177,36.0,1,12.0,25.0,40.0,14.0,[],['双一流'],...,832.00,3,360.0,540.0,0,0,2.0,[],[双一流],0
3,8a69c4e9824fd55f018258b725082f07,8a69f68e82198e9101822545359e0a0d,28.0,1,5.0,40.0,70.0,12.0,[],[],...,401.40,4,360.0,288.0,0,0,5.0,[],[],1
4,2c920715800e8898018022a768d41cc3,2c9207157f347800017f72dd6e04248b,33.0,1,8.0,20.0,30.0,16.0,['双一流'],[],...,692.00,2,360.0,300.0,0,0,2.0,[双一流],[],1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58499,2c9207157e58b769017e703d68483561,2c9207157dc878e5017ddab2b223521b,32.0,15,8.0,8.0,15.0,12.0,[],[],...,783.50,0,360.0,300.0,3,3,4.0,[],[],1
58500,8a69c4e88262c85901826bc2582c33d0,8a69f61482627e7a0182628269db2b56,32.0,6,9.0,11.0,28.0,14.0,[],[],...,814.50,4,270.0,240.0,3,3,4.0,[],[],1
58501,2c9207157ef7ed7f017f0a990d4d23a0,2c9207157ef7ed16017f0c28fa6c5e48,42.0,1,13.0,40.0,90.0,12.0,"['211', '双一流']",[],...,1141.75,4,360.0,300.0,3,3,4.0,"[211, 双一流]",[],1
58502,8a69d7a68219933d01821fc0c69843ea,2c9207158114fb2901815c03bb257372,30.0,1,6.0,20.0,35.0,14.0,[],[],...,501.50,4,600.0,247.0,3,3,4.0,[],[],1


In [20]:
# 首先把expectlocation_range_cv和location_range_jd转成列表
def school_lables_to_list(series, location_col):
    location_labels = series[location_col]
    location_labels_list = [item.strip('\'') for item in location_labels.strip('[]').replace(' ', '').split(',')]
    if location_labels_list[0] == '':
        return []
    return location_labels_list

sample_train['location_range_jd_list'] = sample_train.apply(school_lables_to_list, axis=1, args=('location_range_jd', ))
sample_train['expectlocation_range_cv_list'] = sample_train.apply(school_lables_to_list, axis=1, args=('expectlocation_range_cv', ))

sample_train

,cv_id,jd_id,age,recruit_number,workYear,min_month_salary,max_month_salary,pay_months,school_labels_cv,school_labels_jd,...,parsed_desiredSalary,parsed_currentSalary,da_label,rank_label,resume_type,school_labels_cv_list,school_labels_jd_list,school_intersect_cvjd,location_range_jd_list,expectlocation_range_cv_list
0,2c9207158059e82b018067f52ee520f1,2c9207157ef60164017ef604e195022e,26.0,1,4.0,25.0,32.0,16.0,"['211', '双一流']",[],...,180.0,300.0,0,0,2.0,"[211, 双一流]",[],1,[],[未知市]
1,2c920715804c9ef601805374e101570a,2c92071580b3a1010180bda0e1166bfd,46.0,1,23.0,80.0,100.0,12.0,"['985', '211', '双一流']",[],...,360.0,300.0,0,0,2.0,"[985, 211, 双一流]",[],1,[],[未知市]
2,2c9207157ef7ed7f017f075fc6640e79,2c9207157d2f0ec5017d5a854c143177,36.0,1,12.0,25.0,40.0,14.0,[],['双一流'],...,360.0,540.0,0,0,2.0,[],[双一流],0,[北京市],[未知市]
3,8a69c4e9824fd55f018258b725082f07,8a69f68e82198e9101822545359e0a0d,28.0,1,5.0,40.0,70.0,12.0,[],[],...,360.0,288.0,0,0,5.0,[],[],1,[],"[广州市, 深圳市]"
4,2c920715800e8898018022a768d41cc3,2c9207157f347800017f72dd6e04248b,33.0,1,8.0,20.0,30.0,16.0,['双一流'],[],...,360.0,300.0,0,0,2.0,[双一流],[],1,[],[未知市]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58499,2c9207157e58b769017e703d68483561,2c9207157dc878e5017ddab2b223521b,32.0,15,8.0,8.0,15.0,12.0,[],[],...,360.0,300.0,3,3,4.0,[],[],1,[],[未知市]
58500,8a69c4e88262c85901826bc2582c33d0,8a69f61482627e7a0182628269db2b56,32.0,6,9.0,11.0,28.0,14.0,[],[],...,270.0,240.0,3,3,4.0,[],[],1,[],[南京市]
58501,2c9207157ef7ed7f017f0a990d4d23a0,2c9207157ef7ed16017f0c28fa6c5e48,42.0,1,13.0,40.0,90.0,12.0,"['211', '双一流']",[],...,360.0,300.0,3,3,4.0,"[211, 双一流]",[],1,[],[未知市]
58502,8a69d7a68219933d01821fc0c69843ea,2c9207158114fb2901815c03bb257372,30.0,1,6.0,20.0,35.0,14.0,[],[],...,600.0,247.0,3,3,4.0,[],[],1,[],[上海市]


In [21]:
# location交叉
def location_intersect_cvjd(series, location_labels_cv, location_labels_jd):
    location_labels_cv = series[location_labels_cv]
    location_labels_jd = series[location_labels_jd]
    # 如果jd没有地域
    if not location_labels_jd:
        return 1
    if set(location_labels_cv).intersection(set(location_labels_jd)):
        return 1
    return 0

sample_train['location_intersect_cvjd'] = sample_train.apply(location_intersect_cvjd, axis=1, args=('expectlocation_range_cv_list', 'location_range_jd_list'))
sample_train

,cv_id,jd_id,age,recruit_number,workYear,min_month_salary,max_month_salary,pay_months,school_labels_cv,school_labels_jd,...,parsed_currentSalary,da_label,rank_label,resume_type,school_labels_cv_list,school_labels_jd_list,school_intersect_cvjd,location_range_jd_list,expectlocation_range_cv_list,location_intersect_cvjd
0,2c9207158059e82b018067f52ee520f1,2c9207157ef60164017ef604e195022e,26.0,1,4.0,25.0,32.0,16.0,"['211', '双一流']",[],...,300.0,0,0,2.0,"[211, 双一流]",[],1,[],[未知市],1
1,2c920715804c9ef601805374e101570a,2c92071580b3a1010180bda0e1166bfd,46.0,1,23.0,80.0,100.0,12.0,"['985', '211', '双一流']",[],...,300.0,0,0,2.0,"[985, 211, 双一流]",[],1,[],[未知市],1
2,2c9207157ef7ed7f017f075fc6640e79,2c9207157d2f0ec5017d5a854c143177,36.0,1,12.0,25.0,40.0,14.0,[],['双一流'],...,540.0,0,0,2.0,[],[双一流],0,[北京市],[未知市],0
3,8a69c4e9824fd55f018258b725082f07,8a69f68e82198e9101822545359e0a0d,28.0,1,5.0,40.0,70.0,12.0,[],[],...,288.0,0,0,5.0,[],[],1,[],"[广州市, 深圳市]",1
4,2c920715800e8898018022a768d41cc3,2c9207157f347800017f72dd6e04248b,33.0,1,8.0,20.0,30.0,16.0,['双一流'],[],...,300.0,0,0,2.0,[双一流],[],1,[],[未知市],1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58499,2c9207157e58b769017e703d68483561,2c9207157dc878e5017ddab2b223521b,32.0,15,8.0,8.0,15.0,12.0,[],[],...,300.0,3,3,4.0,[],[],1,[],[未知市],1
58500,8a69c4e88262c85901826bc2582c33d0,8a69f61482627e7a0182628269db2b56,32.0,6,9.0,11.0,28.0,14.0,[],[],...,240.0,3,3,4.0,[],[],1,[],[南京市],1
58501,2c9207157ef7ed7f017f0a990d4d23a0,2c9207157ef7ed16017f0c28fa6c5e48,42.0,1,13.0,40.0,90.0,12.0,"['211', '双一流']",[],...,300.0,3,3,4.0,"[211, 双一流]",[],1,[],[未知市],1
58502,8a69d7a68219933d01821fc0c69843ea,2c9207158114fb2901815c03bb257372,30.0,1,6.0,20.0,35.0,14.0,[],[],...,247.0,3,3,4.0,[],[],1,[],[上海市],1


In [22]:
# degree交叉
def degree_match_cvjd(series, degree_index_cv, degree_index_jd):
    degree_index_cv = series['degree_index_cv']
    degree_index_jd = series['degree_index_jd']
    if degree_index_cv >= degree_index_jd:
        return 1
    else:
        return 0

sample_train['degree_match_cvjd'] = sample_train.apply(degree_match_cvjd, axis=1, args=('degree_index_cv', 'degree_index_jd'))
sample_train

,cv_id,jd_id,age,recruit_number,workYear,min_month_salary,max_month_salary,pay_months,school_labels_cv,school_labels_jd,...,da_label,rank_label,resume_type,school_labels_cv_list,school_labels_jd_list,school_intersect_cvjd,location_range_jd_list,expectlocation_range_cv_list,location_intersect_cvjd,degree_match_cvjd
0,2c9207158059e82b018067f52ee520f1,2c9207157ef60164017ef604e195022e,26.0,1,4.0,25.0,32.0,16.0,"['211', '双一流']",[],...,0,0,2.0,"[211, 双一流]",[],1,[],[未知市],1,1
1,2c920715804c9ef601805374e101570a,2c92071580b3a1010180bda0e1166bfd,46.0,1,23.0,80.0,100.0,12.0,"['985', '211', '双一流']",[],...,0,0,2.0,"[985, 211, 双一流]",[],1,[],[未知市],1,1
2,2c9207157ef7ed7f017f075fc6640e79,2c9207157d2f0ec5017d5a854c143177,36.0,1,12.0,25.0,40.0,14.0,[],['双一流'],...,0,0,2.0,[],[双一流],0,[北京市],[未知市],0,1
3,8a69c4e9824fd55f018258b725082f07,8a69f68e82198e9101822545359e0a0d,28.0,1,5.0,40.0,70.0,12.0,[],[],...,0,0,5.0,[],[],1,[],"[广州市, 深圳市]",1,1
4,2c920715800e8898018022a768d41cc3,2c9207157f347800017f72dd6e04248b,33.0,1,8.0,20.0,30.0,16.0,['双一流'],[],...,0,0,2.0,[双一流],[],1,[],[未知市],1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58499,2c9207157e58b769017e703d68483561,2c9207157dc878e5017ddab2b223521b,32.0,15,8.0,8.0,15.0,12.0,[],[],...,3,3,4.0,[],[],1,[],[未知市],1,1
58500,8a69c4e88262c85901826bc2582c33d0,8a69f61482627e7a0182628269db2b56,32.0,6,9.0,11.0,28.0,14.0,[],[],...,3,3,4.0,[],[],1,[],[南京市],1,1
58501,2c9207157ef7ed7f017f0a990d4d23a0,2c9207157ef7ed16017f0c28fa6c5e48,42.0,1,13.0,40.0,90.0,12.0,"['211', '双一流']",[],...,3,3,4.0,"[211, 双一流]",[],1,[],[未知市],1,1
58502,8a69d7a68219933d01821fc0c69843ea,2c9207158114fb2901815c03bb257372,30.0,1,6.0,20.0,35.0,14.0,[],[],...,3,3,4.0,[],[],1,[],[上海市],1,1


In [23]:
sample_train['rank_label'].value_counts()

1    20266
2    17923
0    13120
3     7195
Name: rank_label, dtype: int64

In [24]:
# 生成模型训练label
def model_label_gen(series, rank_label):
    rank_label = series[rank_label]
    if rank_label in [0, 1]:
        return 0
    return 1
sample_train['model_label'] = sample_train.apply(model_label_gen, axis=1, args=('rank_label', ))
sample_train

,cv_id,jd_id,age,recruit_number,workYear,min_month_salary,max_month_salary,pay_months,school_labels_cv,school_labels_jd,...,rank_label,resume_type,school_labels_cv_list,school_labels_jd_list,school_intersect_cvjd,location_range_jd_list,expectlocation_range_cv_list,location_intersect_cvjd,degree_match_cvjd,model_label
0,2c9207158059e82b018067f52ee520f1,2c9207157ef60164017ef604e195022e,26.0,1,4.0,25.0,32.0,16.0,"['211', '双一流']",[],...,0,2.0,"[211, 双一流]",[],1,[],[未知市],1,1,0
1,2c920715804c9ef601805374e101570a,2c92071580b3a1010180bda0e1166bfd,46.0,1,23.0,80.0,100.0,12.0,"['985', '211', '双一流']",[],...,0,2.0,"[985, 211, 双一流]",[],1,[],[未知市],1,1,0
2,2c9207157ef7ed7f017f075fc6640e79,2c9207157d2f0ec5017d5a854c143177,36.0,1,12.0,25.0,40.0,14.0,[],['双一流'],...,0,2.0,[],[双一流],0,[北京市],[未知市],0,1,0
3,8a69c4e9824fd55f018258b725082f07,8a69f68e82198e9101822545359e0a0d,28.0,1,5.0,40.0,70.0,12.0,[],[],...,0,5.0,[],[],1,[],"[广州市, 深圳市]",1,1,0
4,2c920715800e8898018022a768d41cc3,2c9207157f347800017f72dd6e04248b,33.0,1,8.0,20.0,30.0,16.0,['双一流'],[],...,0,2.0,[双一流],[],1,[],[未知市],1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58499,2c9207157e58b769017e703d68483561,2c9207157dc878e5017ddab2b223521b,32.0,15,8.0,8.0,15.0,12.0,[],[],...,3,4.0,[],[],1,[],[未知市],1,1,1
58500,8a69c4e88262c85901826bc2582c33d0,8a69f61482627e7a0182628269db2b56,32.0,6,9.0,11.0,28.0,14.0,[],[],...,3,4.0,[],[],1,[],[南京市],1,1,1
58501,2c9207157ef7ed7f017f0a990d4d23a0,2c9207157ef7ed16017f0c28fa6c5e48,42.0,1,13.0,40.0,90.0,12.0,"['211', '双一流']",[],...,3,4.0,"[211, 双一流]",[],1,[],[未知市],1,1,1
58502,8a69d7a68219933d01821fc0c69843ea,2c9207158114fb2901815c03bb257372,30.0,1,6.0,20.0,35.0,14.0,[],[],...,3,4.0,[],[],1,[],[上海市],1,1,1


In [25]:
sample_train['model_label'].value_counts()

0    33386
1    25118
Name: model_label, dtype: int64

In [ ]:
# 将jd中的月薪转成年薪
def multiply_cols_fun(series, col1, col2):
    col1_value = series[col1]
    col2_value = series[col2]
    return col1_value * col2_value

sample_train['min_annual_salary'] = sample_train.apply(multiply_cols_fun, axis=1, args=('min_month_salary', 'pay_months'))
sample_train['max_annual_salary'] = sample_train.apply(multiply_cols_fun, axis=1, args=('max_month_salary', 'pay_months'))

In [ ]:
# 先把workyear_range_jd转成列表
def school_lables_to_list(series, location_col):
    location_labels = series[location_col]
    location_labels_list = [item.strip('\'') for item in location_labels.strip('[]').replace(' ', '').split(',')]
    first = int(location_labels_list[0])
    second = int(location_labels_list[1])
    return [first, second]

sample_train['workyear_range_jd_list'] = sample_train.apply(school_lables_to_list, axis=1, args=('workyear_range_jd', ))

sample_train[['workyear_range_jd', 'workyear_range_jd_list']]

In [ ]:
# work year match
def workYear_match_fun(series, workyear_range_jd_list, workYear):
    workyear_range = series[workyear_range_jd_list]
    workYear = series[workYear]
    if workyear_range[0] <= workYear <= workyear_range[1]:
        return 1
    else:
        return 0
    
sample_train['workyear_match_cvjd'] = sample_train.apply(workYear_match_fun, axis=1, args=('workyear_range_jd_list', 'workYear'))

In [ ]:
# 好学校
def keyU_label_fun(series, school_labels_cv_list):
    school_labels = series[school_labels_cv_list]
    if school_labels:
        return 1
    else:
        return 0
sample_train['keyU'] = sample_train.apply(keyU_label_fun, axis=1, args=('school_labels_cv_list',))

In [ ]:
# tfidf_pca转为向量（保存为csv的时候会由向量变成字符串）
def school_lables_to_list(series, school_col):
    school_labels = series[school_col]
    school_labels_list = [float(item) for item in school_labels.strip('[]').replace(' ', '').split(',')]
    if school_labels_list[0] == '':
        return []
    return school_labels_list
sample_train['description_tfidf_pca'] = sample_train.apply(school_lables_to_list, axis=1, args=('description_tfidf_pca', ))
sample_train['position_tfidf_pca'] = sample_train.apply(school_lables_to_list, axis=1, args=('position_tfidf_pca', ))
sample_train['requirement_tfidf_pca'] = sample_train.apply(school_lables_to_list, axis=1, args=('requirement_tfidf_pca', ))
sample_train['kills_tfidf_pca'] = sample_train.apply(school_lables_to_list, axis=1, args=('kills_tfidf_pca', ))
sample_train['title_category_tags_tfidf_pca'] = sample_train.apply(school_lables_to_list, axis=1, args=('title_category_tags_tfidf_pca', ))

In [ ]:
# 几个list如何处理
# parsed_currentSalary特征不使用
# min_month_salary和max_month_salary分别乘以pay_months，得到年薪数据，以k为单位。已处理
# salary特征可信度不高，清华模型就是简单的输入deep侧，除此以外没有任何处理。
COLUMNS = ['age', 'recruit_number', 'workYear', 'min_annual_salary', 'max_annual_salary', 
            'degree_index_cv', 'degree_index_jd', 'gender_index_cv', 'job_intentstatus_cv',
            'description_tfidf_pca', 'position_tfidf_pca', 'requirement_tfidf_pca', 'kills_tfidf_pca', 'title_category_tags_tfidf_pca',
            'work_duration_mean', 'equal_job', 'parsed_desiredSalary', 'school_intersect_cvjd', 'location_intersect_cvjd',
            'degree_match_cvjd', 'workyear_match_cvjd', 'keyU']

LABEL_COLUMN = "model_label"

# 离散特征
CATEGORICAL_COLUMNS = ['degree_index_cv', 'degree_index_jd', 'gender_index_cv', 'job_intentstatus_cv', 'school_intersect_cvjd',
                        'location_intersect_cvjd', 'degree_match_cvjd', 'workyear_match_cvjd', 'keyU']

# 文本向量特征
ARRAY_COLUMNS = ['description_tfidf_pca', 'position_tfidf_pca', 'requirement_tfidf_pca', 'kills_tfidf_pca', 'title_category_tags_tfidf_pca']

# 连续特征
CONTINUOUS_COLUMNS = ['age', 'recruit_number', 'workYear', 'min_annual_salary', 'max_annual_salary', 
                    'work_duration_mean', 'equal_job', 'parsed_desiredSalary', ]


In [ ]:
COLUMNS.sort()
print(COLUMNS)
print(len(COLUMNS))

In [ ]:
all_columns = ['cv_id', 'jd_id', 'model_label'] + COLUMNS
print(all_columns)
print(len(all_columns))

In [ ]:
sample_train[all_columns].to_pickle('../data_20220831/sample_train_processed_processed.pkl')

# sample_test缺失值填充


In [26]:
import numpy as np
import pandas as pd

sample_test = pd.read_csv('../data_20220831/sample_test.csv').drop('Unnamed: 0', axis=1)

In [27]:
sample_test.columns

Index(['cv_id', 'jd_id', 'age', 'recruit_number', 'workYear',
       'min_month_salary', 'max_month_salary', 'pay_months',
       'school_labels_cv', 'school_labels_jd', 'expectlocation_range_cv',
       'location_range_jd', 'degree_index_cv', 'degree_index_jd',
       'workyear_range_jd', 'gender_index_cv', 'job_intentstatus_cv',
       'description_tfidf_pca', 'position_tfidf_pca', 'requirement_tfidf_pca',
       'kills_tfidf_pca', 'title_category_tags_tfidf_pca',
       'work_duration_mean', 'equal_job', 'parsed_desiredSalary',
       'parsed_currentSalary', 'da_label', 'rank_label', 'resume_type'],
      dtype='object')

In [28]:
sample_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14627 entries, 0 to 14626
Data columns (total 29 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   cv_id                          14627 non-null  object 
 1   jd_id                          14627 non-null  object 
 2   age                            13340 non-null  float64
 3   recruit_number                 14627 non-null  int64  
 4   workYear                       13586 non-null  float64
 5   min_month_salary               14137 non-null  float64
 6   max_month_salary               14137 non-null  float64
 7   pay_months                     14137 non-null  float64
 8   school_labels_cv               14627 non-null  object 
 9   school_labels_jd               14627 non-null  object 
 10  expectlocation_range_cv        14627 non-null  object 
 11  location_range_jd              14627 non-null  object 
 12  degree_index_cv                14627 non-null 

In [29]:
sample_test

,cv_id,jd_id,age,recruit_number,workYear,min_month_salary,max_month_salary,pay_months,school_labels_cv,school_labels_jd,...,requirement_tfidf_pca,kills_tfidf_pca,title_category_tags_tfidf_pca,work_duration_mean,equal_job,parsed_desiredSalary,parsed_currentSalary,da_label,rank_label,resume_type
0,2c9207157ef7ed7f017efd02e21e4e49,2c9207157d2f0ec5017d5a7504c72f8b,31.0,1,9.0,NaN,NaN,NaN,['双一流'],['双一流'],...,"[-0.1419118611278064, -0.1529188641300954, -0....","[0.2864471657745184, 0.2252494388057755, -0.22...","[-0.2999534601671801, 0.1675461791724773, 0.36...",1537.500000,3,-1,-1,0,0,2.0
1,2c920715802399a801804bdec5c34543,2c920715800e886301801791ef1b53ee,39.0,1,7.0,12.0,13.0,12.0,[],[],...,"[-0.1076417254917609, 0.1969000746662075, 3.28...","[-0.0654732692868987, -0.0805561760755879, 0.0...","[-0.0450986330342915, -0.2783286045295846, -0....",386.166667,1,600,360,0,0,2.0
2,2c92071581c8d04f0181cb3aa9f4639c,2c92071581c8c61e0181cc4416582615,39.0,1,5.0,30.0,50.0,12.0,['双一流'],[],...,"[-0.0935522486058823, 0.0394952248909006, -0.0...","[0.0760272834665939, -0.1730278473983863, 0.09...","[0.0088934337229539, -0.0552129385459116, -0.0...",509.000000,0,-1,-1,0,0,5.0
3,2c920715802399a801804ba70b163677,2c920715800e886301801791ef1b53ee,37.0,1,9.0,12.0,13.0,12.0,[],[],...,"[-0.1076417254917609, 0.1969000746662075, 3.28...","[0.039229463903765, -0.0956845975092942, 0.059...","[-0.0450986330342915, -0.2783286045295846, -0....",1461.000000,1,-1,-1,0,0,2.0
4,2c92071580b3a89c0180d1cdb36c3c4b,2c920715816798cb01817f99f0341751,34.0,1,12.0,15.0,35.0,24.0,"['211', '双一流']","['211', '985', 'qs100']",...,"[-0.0296257942655244, -0.095514068279019, -0.2...","[-0.0599022710215368, 0.2894064718007514, 0.08...","[-0.343357678493863, 0.250890015200693, -0.097...",834.000000,7,840,-1,0,0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14622,2c92071581d380c20181ede1f6b4401f,2c92071581d380a00181dccc2ac54686,46.0,1,23.0,10.0,30.0,12.0,"['985', '211', '双一流']",[],...,"[-0.0635984771163071, 0.1566419781432294, -0.0...","[0.2121591919281735, -0.0973096944987941, 0.18...","[0.0292751648590925, -0.0946458180860336, -0.0...",1564.600000,1,-1,600,3,3,4.0
14623,2c9207157dec317f017e0e75e24d4ab8,2c9207157dec2c34017e05671af21551,36.0,1,8.0,8.0,12.0,12.0,[],[],...,"[-0.0374584565352739, 0.0564068476779357, 0.04...","[0.1461731824810366, -0.153381590345665, 0.189...","[-0.0458219190722429, -0.0389132984183317, -0....",1156.333333,2,182,-1,3,3,4.0
14624,2c92071581d380c20181d7a76b390856,2c92071581cdd33b0181d224cbdf7b06,23.0,1,3.0,15.0,25.0,13.0,[],[],...,"[-0.0930706771115408, -0.0359898261643764, -0....","[-0.1159695809393811, -0.0562285602178969, -0....","[-0.0527260069646775, -0.0171684419689047, -0....",548.000000,2,168,108,3,3,4.0
14625,2c920715816798dd01816b5b135e64b2,2c9207158114fb29018148007d475ade,23.0,1,4.0,6.0,8.0,12.0,[],[],...,"[-0.0113261949341071, 0.1086270271668797, 0.00...","[-0.0970040303205414, -0.0440447197647652, 0.0...","[0.0246964705746208, -0.0857444778704595, -0.0...",426.333333,3,84,-1,3,3,4.0


In [30]:
# 中位数填充
from sklearn.impute import SimpleImputer

imp = SimpleImputer(missing_values=np.nan, strategy='median')
sample_test['age'] = imp.fit_transform(sample_test[['age']])
sample_test['workYear'] = imp.fit_transform(sample_test[['workYear']])
sample_test['min_month_salary'] = imp.fit_transform(sample_test[['min_month_salary']])
sample_test['max_month_salary'] = imp.fit_transform(sample_test[['max_month_salary']])
sample_test['pay_months'] = imp.fit_transform(sample_test[['pay_months']])
sample_test['work_duration_mean'] = imp.fit_transform(sample_test[['work_duration_mean']])

imp = SimpleImputer(missing_values=-1, strategy='median')
# 这两个只能选一个接入模型
sample_test['parsed_desiredSalary'] = imp.fit_transform(sample_test[['parsed_desiredSalary']])
sample_test['parsed_currentSalary'] = imp.fit_transform(sample_test[['parsed_currentSalary']])
sample_test


,cv_id,jd_id,age,recruit_number,workYear,min_month_salary,max_month_salary,pay_months,school_labels_cv,school_labels_jd,...,requirement_tfidf_pca,kills_tfidf_pca,title_category_tags_tfidf_pca,work_duration_mean,equal_job,parsed_desiredSalary,parsed_currentSalary,da_label,rank_label,resume_type
0,2c9207157ef7ed7f017efd02e21e4e49,2c9207157d2f0ec5017d5a7504c72f8b,31.0,1,9.0,25.0,40.0,14.0,['双一流'],['双一流'],...,"[-0.1419118611278064, -0.1529188641300954, -0....","[0.2864471657745184, 0.2252494388057755, -0.22...","[-0.2999534601671801, 0.1675461791724773, 0.36...",1537.500000,3,360.0,300.0,0,0,2.0
1,2c920715802399a801804bdec5c34543,2c920715800e886301801791ef1b53ee,39.0,1,7.0,12.0,13.0,12.0,[],[],...,"[-0.1076417254917609, 0.1969000746662075, 3.28...","[-0.0654732692868987, -0.0805561760755879, 0.0...","[-0.0450986330342915, -0.2783286045295846, -0....",386.166667,1,600.0,360.0,0,0,2.0
2,2c92071581c8d04f0181cb3aa9f4639c,2c92071581c8c61e0181cc4416582615,39.0,1,5.0,30.0,50.0,12.0,['双一流'],[],...,"[-0.0935522486058823, 0.0394952248909006, -0.0...","[0.0760272834665939, -0.1730278473983863, 0.09...","[0.0088934337229539, -0.0552129385459116, -0.0...",509.000000,0,360.0,300.0,0,0,5.0
3,2c920715802399a801804ba70b163677,2c920715800e886301801791ef1b53ee,37.0,1,9.0,12.0,13.0,12.0,[],[],...,"[-0.1076417254917609, 0.1969000746662075, 3.28...","[0.039229463903765, -0.0956845975092942, 0.059...","[-0.0450986330342915, -0.2783286045295846, -0....",1461.000000,1,360.0,300.0,0,0,2.0
4,2c92071580b3a89c0180d1cdb36c3c4b,2c920715816798cb01817f99f0341751,34.0,1,12.0,15.0,35.0,24.0,"['211', '双一流']","['211', '985', 'qs100']",...,"[-0.0296257942655244, -0.095514068279019, -0.2...","[-0.0599022710215368, 0.2894064718007514, 0.08...","[-0.343357678493863, 0.250890015200693, -0.097...",834.000000,7,840.0,300.0,0,0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14622,2c92071581d380c20181ede1f6b4401f,2c92071581d380a00181dccc2ac54686,46.0,1,23.0,10.0,30.0,12.0,"['985', '211', '双一流']",[],...,"[-0.0635984771163071, 0.1566419781432294, -0.0...","[0.2121591919281735, -0.0973096944987941, 0.18...","[0.0292751648590925, -0.0946458180860336, -0.0...",1564.600000,1,360.0,600.0,3,3,4.0
14623,2c9207157dec317f017e0e75e24d4ab8,2c9207157dec2c34017e05671af21551,36.0,1,8.0,8.0,12.0,12.0,[],[],...,"[-0.0374584565352739, 0.0564068476779357, 0.04...","[0.1461731824810366, -0.153381590345665, 0.189...","[-0.0458219190722429, -0.0389132984183317, -0....",1156.333333,2,182.0,300.0,3,3,4.0
14624,2c92071581d380c20181d7a76b390856,2c92071581cdd33b0181d224cbdf7b06,23.0,1,3.0,15.0,25.0,13.0,[],[],...,"[-0.0930706771115408, -0.0359898261643764, -0....","[-0.1159695809393811, -0.0562285602178969, -0....","[-0.0527260069646775, -0.0171684419689047, -0....",548.000000,2,168.0,108.0,3,3,4.0
14625,2c920715816798dd01816b5b135e64b2,2c9207158114fb29018148007d475ade,23.0,1,4.0,6.0,8.0,12.0,[],[],...,"[-0.0113261949341071, 0.1086270271668797, 0.00...","[-0.0970040303205414, -0.0440447197647652, 0.0...","[0.0246964705746208, -0.0857444778704595, -0.0...",426.333333,3,84.0,300.0,3,3,4.0


# sample_test手动cross

In [32]:
# 首先把school_labels_cv和school_labels_jd转成列表
def school_lables_to_list(series, school_col):
    school_labels = series[school_col]
    school_labels_list = [item.strip('\'') for item in school_labels.strip('[]').replace(' ', '').split(',')]
    if school_labels_list[0] == '':
        return []
    return school_labels_list
sample_test['school_labels_cv_list'] = sample_test.apply(school_lables_to_list, axis=1, args=('school_labels_cv', ))
sample_test['school_labels_jd_list'] = sample_test.apply(school_lables_to_list, axis=1, args=('school_labels_jd', ))

sample_test

,cv_id,jd_id,age,recruit_number,workYear,min_month_salary,max_month_salary,pay_months,school_labels_cv,school_labels_jd,...,title_category_tags_tfidf_pca,work_duration_mean,equal_job,parsed_desiredSalary,parsed_currentSalary,da_label,rank_label,resume_type,school_labels_cv_list,school_labels_jd_list
0,2c9207157ef7ed7f017efd02e21e4e49,2c9207157d2f0ec5017d5a7504c72f8b,31.0,1,9.0,25.0,40.0,14.0,['双一流'],['双一流'],...,"[-0.2999534601671801, 0.1675461791724773, 0.36...",1537.500000,3,360.0,300.0,0,0,2.0,[双一流],[双一流]
1,2c920715802399a801804bdec5c34543,2c920715800e886301801791ef1b53ee,39.0,1,7.0,12.0,13.0,12.0,[],[],...,"[-0.0450986330342915, -0.2783286045295846, -0....",386.166667,1,600.0,360.0,0,0,2.0,[],[]
2,2c92071581c8d04f0181cb3aa9f4639c,2c92071581c8c61e0181cc4416582615,39.0,1,5.0,30.0,50.0,12.0,['双一流'],[],...,"[0.0088934337229539, -0.0552129385459116, -0.0...",509.000000,0,360.0,300.0,0,0,5.0,[双一流],[]
3,2c920715802399a801804ba70b163677,2c920715800e886301801791ef1b53ee,37.0,1,9.0,12.0,13.0,12.0,[],[],...,"[-0.0450986330342915, -0.2783286045295846, -0....",1461.000000,1,360.0,300.0,0,0,2.0,[],[]
4,2c92071580b3a89c0180d1cdb36c3c4b,2c920715816798cb01817f99f0341751,34.0,1,12.0,15.0,35.0,24.0,"['211', '双一流']","['211', '985', 'qs100']",...,"[-0.343357678493863, 0.250890015200693, -0.097...",834.000000,7,840.0,300.0,0,0,2.0,"[211, 双一流]","[211, 985, qs100]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14622,2c92071581d380c20181ede1f6b4401f,2c92071581d380a00181dccc2ac54686,46.0,1,23.0,10.0,30.0,12.0,"['985', '211', '双一流']",[],...,"[0.0292751648590925, -0.0946458180860336, -0.0...",1564.600000,1,360.0,600.0,3,3,4.0,"[985, 211, 双一流]",[]
14623,2c9207157dec317f017e0e75e24d4ab8,2c9207157dec2c34017e05671af21551,36.0,1,8.0,8.0,12.0,12.0,[],[],...,"[-0.0458219190722429, -0.0389132984183317, -0....",1156.333333,2,182.0,300.0,3,3,4.0,[],[]
14624,2c92071581d380c20181d7a76b390856,2c92071581cdd33b0181d224cbdf7b06,23.0,1,3.0,15.0,25.0,13.0,[],[],...,"[-0.0527260069646775, -0.0171684419689047, -0....",548.000000,2,168.0,108.0,3,3,4.0,[],[]
14625,2c920715816798dd01816b5b135e64b2,2c9207158114fb29018148007d475ade,23.0,1,4.0,6.0,8.0,12.0,[],[],...,"[0.0246964705746208, -0.0857444778704595, -0.0...",426.333333,3,84.0,300.0,3,3,4.0,[],[]


In [33]:
def school_intersect_cvjd(series, school_labels_cv, school_labels_jd):
    school_labels_cv = series[school_labels_cv]
    school_labels_jd = series[school_labels_jd]
    # jd没有学历要求or解析不出来
    if not school_labels_jd:
        return 1
    if set(school_labels_cv).intersection(set(school_labels_jd)):
        return 1
    return 0

sample_test['school_intersect_cvjd'] = sample_test.apply(school_intersect_cvjd, axis=1, args=('school_labels_cv_list', 'school_labels_jd_list'))
sample_test

,cv_id,jd_id,age,recruit_number,workYear,min_month_salary,max_month_salary,pay_months,school_labels_cv,school_labels_jd,...,work_duration_mean,equal_job,parsed_desiredSalary,parsed_currentSalary,da_label,rank_label,resume_type,school_labels_cv_list,school_labels_jd_list,school_intersect_cvjd
0,2c9207157ef7ed7f017efd02e21e4e49,2c9207157d2f0ec5017d5a7504c72f8b,31.0,1,9.0,25.0,40.0,14.0,['双一流'],['双一流'],...,1537.500000,3,360.0,300.0,0,0,2.0,[双一流],[双一流],1
1,2c920715802399a801804bdec5c34543,2c920715800e886301801791ef1b53ee,39.0,1,7.0,12.0,13.0,12.0,[],[],...,386.166667,1,600.0,360.0,0,0,2.0,[],[],1
2,2c92071581c8d04f0181cb3aa9f4639c,2c92071581c8c61e0181cc4416582615,39.0,1,5.0,30.0,50.0,12.0,['双一流'],[],...,509.000000,0,360.0,300.0,0,0,5.0,[双一流],[],1
3,2c920715802399a801804ba70b163677,2c920715800e886301801791ef1b53ee,37.0,1,9.0,12.0,13.0,12.0,[],[],...,1461.000000,1,360.0,300.0,0,0,2.0,[],[],1
4,2c92071580b3a89c0180d1cdb36c3c4b,2c920715816798cb01817f99f0341751,34.0,1,12.0,15.0,35.0,24.0,"['211', '双一流']","['211', '985', 'qs100']",...,834.000000,7,840.0,300.0,0,0,2.0,"[211, 双一流]","[211, 985, qs100]",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14622,2c92071581d380c20181ede1f6b4401f,2c92071581d380a00181dccc2ac54686,46.0,1,23.0,10.0,30.0,12.0,"['985', '211', '双一流']",[],...,1564.600000,1,360.0,600.0,3,3,4.0,"[985, 211, 双一流]",[],1
14623,2c9207157dec317f017e0e75e24d4ab8,2c9207157dec2c34017e05671af21551,36.0,1,8.0,8.0,12.0,12.0,[],[],...,1156.333333,2,182.0,300.0,3,3,4.0,[],[],1
14624,2c92071581d380c20181d7a76b390856,2c92071581cdd33b0181d224cbdf7b06,23.0,1,3.0,15.0,25.0,13.0,[],[],...,548.000000,2,168.0,108.0,3,3,4.0,[],[],1
14625,2c920715816798dd01816b5b135e64b2,2c9207158114fb29018148007d475ade,23.0,1,4.0,6.0,8.0,12.0,[],[],...,426.333333,3,84.0,300.0,3,3,4.0,[],[],1


In [34]:
# 首先把expectlocation_range_cv和location_range_jd转成列表
def school_lables_to_list(series, location_col):
    location_labels = series[location_col]
    location_labels_list = [item.strip('\'') for item in location_labels.strip('[]').replace(' ', '').split(',')]
    if location_labels_list[0] == '':
        return []
    return location_labels_list

sample_test['location_range_jd_list'] = sample_test.apply(school_lables_to_list, axis=1, args=('location_range_jd', ))
sample_test['expectlocation_range_cv_list'] = sample_test.apply(school_lables_to_list, axis=1, args=('expectlocation_range_cv', ))

sample_test

,cv_id,jd_id,age,recruit_number,workYear,min_month_salary,max_month_salary,pay_months,school_labels_cv,school_labels_jd,...,parsed_desiredSalary,parsed_currentSalary,da_label,rank_label,resume_type,school_labels_cv_list,school_labels_jd_list,school_intersect_cvjd,location_range_jd_list,expectlocation_range_cv_list
0,2c9207157ef7ed7f017efd02e21e4e49,2c9207157d2f0ec5017d5a7504c72f8b,31.0,1,9.0,25.0,40.0,14.0,['双一流'],['双一流'],...,360.0,300.0,0,0,2.0,[双一流],[双一流],1,"[北京市, 上海市]",[北京市]
1,2c920715802399a801804bdec5c34543,2c920715800e886301801791ef1b53ee,39.0,1,7.0,12.0,13.0,12.0,[],[],...,600.0,360.0,0,0,2.0,[],[],1,[],[未知市]
2,2c92071581c8d04f0181cb3aa9f4639c,2c92071581c8c61e0181cc4416582615,39.0,1,5.0,30.0,50.0,12.0,['双一流'],[],...,360.0,300.0,0,0,5.0,[双一流],[],1,[],[未知市]
3,2c920715802399a801804ba70b163677,2c920715800e886301801791ef1b53ee,37.0,1,9.0,12.0,13.0,12.0,[],[],...,360.0,300.0,0,0,2.0,[],[],1,[],[未知市]
4,2c92071580b3a89c0180d1cdb36c3c4b,2c920715816798cb01817f99f0341751,34.0,1,12.0,15.0,35.0,24.0,"['211', '双一流']","['211', '985', 'qs100']",...,840.0,300.0,0,0,2.0,"[211, 双一流]","[211, 985, qs100]",1,[],[未知市]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14622,2c92071581d380c20181ede1f6b4401f,2c92071581d380a00181dccc2ac54686,46.0,1,23.0,10.0,30.0,12.0,"['985', '211', '双一流']",[],...,360.0,600.0,3,3,4.0,"[985, 211, 双一流]",[],1,[],[未知市]
14623,2c9207157dec317f017e0e75e24d4ab8,2c9207157dec2c34017e05671af21551,36.0,1,8.0,8.0,12.0,12.0,[],[],...,182.0,300.0,3,3,4.0,[],[],1,[],[未知市]
14624,2c92071581d380c20181d7a76b390856,2c92071581cdd33b0181d224cbdf7b06,23.0,1,3.0,15.0,25.0,13.0,[],[],...,168.0,108.0,3,3,4.0,[],[],1,[],[深圳市]
14625,2c920715816798dd01816b5b135e64b2,2c9207158114fb29018148007d475ade,23.0,1,4.0,6.0,8.0,12.0,[],[],...,84.0,300.0,3,3,4.0,[],[],1,[],[未知市]


In [35]:
def location_intersect_cvjd(series, location_labels_cv, location_labels_jd):
    location_labels_cv = series[location_labels_cv]
    location_labels_jd = series[location_labels_jd]
    # 如果jd没有地域
    if not location_labels_jd:
        return 1
    if set(location_labels_cv).intersection(set(location_labels_jd)):
        return 1
    return 0

sample_test['location_intersect_cvjd'] = sample_test.apply(location_intersect_cvjd, axis=1, args=('expectlocation_range_cv_list', 'location_range_jd_list'))
sample_test

,cv_id,jd_id,age,recruit_number,workYear,min_month_salary,max_month_salary,pay_months,school_labels_cv,school_labels_jd,...,parsed_currentSalary,da_label,rank_label,resume_type,school_labels_cv_list,school_labels_jd_list,school_intersect_cvjd,location_range_jd_list,expectlocation_range_cv_list,location_intersect_cvjd
0,2c9207157ef7ed7f017efd02e21e4e49,2c9207157d2f0ec5017d5a7504c72f8b,31.0,1,9.0,25.0,40.0,14.0,['双一流'],['双一流'],...,300.0,0,0,2.0,[双一流],[双一流],1,"[北京市, 上海市]",[北京市],1
1,2c920715802399a801804bdec5c34543,2c920715800e886301801791ef1b53ee,39.0,1,7.0,12.0,13.0,12.0,[],[],...,360.0,0,0,2.0,[],[],1,[],[未知市],1
2,2c92071581c8d04f0181cb3aa9f4639c,2c92071581c8c61e0181cc4416582615,39.0,1,5.0,30.0,50.0,12.0,['双一流'],[],...,300.0,0,0,5.0,[双一流],[],1,[],[未知市],1
3,2c920715802399a801804ba70b163677,2c920715800e886301801791ef1b53ee,37.0,1,9.0,12.0,13.0,12.0,[],[],...,300.0,0,0,2.0,[],[],1,[],[未知市],1
4,2c92071580b3a89c0180d1cdb36c3c4b,2c920715816798cb01817f99f0341751,34.0,1,12.0,15.0,35.0,24.0,"['211', '双一流']","['211', '985', 'qs100']",...,300.0,0,0,2.0,"[211, 双一流]","[211, 985, qs100]",1,[],[未知市],1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14622,2c92071581d380c20181ede1f6b4401f,2c92071581d380a00181dccc2ac54686,46.0,1,23.0,10.0,30.0,12.0,"['985', '211', '双一流']",[],...,600.0,3,3,4.0,"[985, 211, 双一流]",[],1,[],[未知市],1
14623,2c9207157dec317f017e0e75e24d4ab8,2c9207157dec2c34017e05671af21551,36.0,1,8.0,8.0,12.0,12.0,[],[],...,300.0,3,3,4.0,[],[],1,[],[未知市],1
14624,2c92071581d380c20181d7a76b390856,2c92071581cdd33b0181d224cbdf7b06,23.0,1,3.0,15.0,25.0,13.0,[],[],...,108.0,3,3,4.0,[],[],1,[],[深圳市],1
14625,2c920715816798dd01816b5b135e64b2,2c9207158114fb29018148007d475ade,23.0,1,4.0,6.0,8.0,12.0,[],[],...,300.0,3,3,4.0,[],[],1,[],[未知市],1


In [36]:
def degree_match_cvjd(series, degree_index_cv, degree_index_jd):
    degree_index_cv = series['degree_index_cv']
    degree_index_jd = series['degree_index_jd']
    if degree_index_cv >= degree_index_jd:
        return 1
    else:
        return 0

sample_test['degree_match_cvjd'] = sample_test.apply(degree_match_cvjd, axis=1, args=('degree_index_cv', 'degree_index_jd'))
sample_test

,cv_id,jd_id,age,recruit_number,workYear,min_month_salary,max_month_salary,pay_months,school_labels_cv,school_labels_jd,...,da_label,rank_label,resume_type,school_labels_cv_list,school_labels_jd_list,school_intersect_cvjd,location_range_jd_list,expectlocation_range_cv_list,location_intersect_cvjd,degree_match_cvjd
0,2c9207157ef7ed7f017efd02e21e4e49,2c9207157d2f0ec5017d5a7504c72f8b,31.0,1,9.0,25.0,40.0,14.0,['双一流'],['双一流'],...,0,0,2.0,[双一流],[双一流],1,"[北京市, 上海市]",[北京市],1,1
1,2c920715802399a801804bdec5c34543,2c920715800e886301801791ef1b53ee,39.0,1,7.0,12.0,13.0,12.0,[],[],...,0,0,2.0,[],[],1,[],[未知市],1,1
2,2c92071581c8d04f0181cb3aa9f4639c,2c92071581c8c61e0181cc4416582615,39.0,1,5.0,30.0,50.0,12.0,['双一流'],[],...,0,0,5.0,[双一流],[],1,[],[未知市],1,1
3,2c920715802399a801804ba70b163677,2c920715800e886301801791ef1b53ee,37.0,1,9.0,12.0,13.0,12.0,[],[],...,0,0,2.0,[],[],1,[],[未知市],1,1
4,2c92071580b3a89c0180d1cdb36c3c4b,2c920715816798cb01817f99f0341751,34.0,1,12.0,15.0,35.0,24.0,"['211', '双一流']","['211', '985', 'qs100']",...,0,0,2.0,"[211, 双一流]","[211, 985, qs100]",1,[],[未知市],1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14622,2c92071581d380c20181ede1f6b4401f,2c92071581d380a00181dccc2ac54686,46.0,1,23.0,10.0,30.0,12.0,"['985', '211', '双一流']",[],...,3,3,4.0,"[985, 211, 双一流]",[],1,[],[未知市],1,1
14623,2c9207157dec317f017e0e75e24d4ab8,2c9207157dec2c34017e05671af21551,36.0,1,8.0,8.0,12.0,12.0,[],[],...,3,3,4.0,[],[],1,[],[未知市],1,1
14624,2c92071581d380c20181d7a76b390856,2c92071581cdd33b0181d224cbdf7b06,23.0,1,3.0,15.0,25.0,13.0,[],[],...,3,3,4.0,[],[],1,[],[深圳市],1,1
14625,2c920715816798dd01816b5b135e64b2,2c9207158114fb29018148007d475ade,23.0,1,4.0,6.0,8.0,12.0,[],[],...,3,3,4.0,[],[],1,[],[未知市],1,0


In [37]:
sample_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14627 entries, 0 to 14626
Data columns (total 36 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   cv_id                          14627 non-null  object 
 1   jd_id                          14627 non-null  object 
 2   age                            14627 non-null  float64
 3   recruit_number                 14627 non-null  int64  
 4   workYear                       14627 non-null  float64
 5   min_month_salary               14627 non-null  float64
 6   max_month_salary               14627 non-null  float64
 7   pay_months                     14627 non-null  float64
 8   school_labels_cv               14627 non-null  object 
 9   school_labels_jd               14627 non-null  object 
 10  expectlocation_range_cv        14627 non-null  object 
 11  location_range_jd              14627 non-null  object 
 12  degree_index_cv                14627 non-null 

In [38]:
sample_test['rank_label'].value_counts()

1    5067
2    4481
0    3280
3    1799
Name: rank_label, dtype: int64

In [39]:
def model_label_gen(series, rank_label):
    rank_label = series[rank_label]
    if rank_label in [0, 1]:
        return 0
    return 1
sample_test['model_label'] = sample_test.apply(model_label_gen, axis=1, args=('rank_label', ))
sample_test

,cv_id,jd_id,age,recruit_number,workYear,min_month_salary,max_month_salary,pay_months,school_labels_cv,school_labels_jd,...,rank_label,resume_type,school_labels_cv_list,school_labels_jd_list,school_intersect_cvjd,location_range_jd_list,expectlocation_range_cv_list,location_intersect_cvjd,degree_match_cvjd,model_label
0,2c9207157ef7ed7f017efd02e21e4e49,2c9207157d2f0ec5017d5a7504c72f8b,31.0,1,9.0,25.0,40.0,14.0,['双一流'],['双一流'],...,0,2.0,[双一流],[双一流],1,"[北京市, 上海市]",[北京市],1,1,0
1,2c920715802399a801804bdec5c34543,2c920715800e886301801791ef1b53ee,39.0,1,7.0,12.0,13.0,12.0,[],[],...,0,2.0,[],[],1,[],[未知市],1,1,0
2,2c92071581c8d04f0181cb3aa9f4639c,2c92071581c8c61e0181cc4416582615,39.0,1,5.0,30.0,50.0,12.0,['双一流'],[],...,0,5.0,[双一流],[],1,[],[未知市],1,1,0
3,2c920715802399a801804ba70b163677,2c920715800e886301801791ef1b53ee,37.0,1,9.0,12.0,13.0,12.0,[],[],...,0,2.0,[],[],1,[],[未知市],1,1,0
4,2c92071580b3a89c0180d1cdb36c3c4b,2c920715816798cb01817f99f0341751,34.0,1,12.0,15.0,35.0,24.0,"['211', '双一流']","['211', '985', 'qs100']",...,0,2.0,"[211, 双一流]","[211, 985, qs100]",1,[],[未知市],1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14622,2c92071581d380c20181ede1f6b4401f,2c92071581d380a00181dccc2ac54686,46.0,1,23.0,10.0,30.0,12.0,"['985', '211', '双一流']",[],...,3,4.0,"[985, 211, 双一流]",[],1,[],[未知市],1,1,1
14623,2c9207157dec317f017e0e75e24d4ab8,2c9207157dec2c34017e05671af21551,36.0,1,8.0,8.0,12.0,12.0,[],[],...,3,4.0,[],[],1,[],[未知市],1,1,1
14624,2c92071581d380c20181d7a76b390856,2c92071581cdd33b0181d224cbdf7b06,23.0,1,3.0,15.0,25.0,13.0,[],[],...,3,4.0,[],[],1,[],[深圳市],1,1,1
14625,2c920715816798dd01816b5b135e64b2,2c9207158114fb29018148007d475ade,23.0,1,4.0,6.0,8.0,12.0,[],[],...,3,4.0,[],[],1,[],[未知市],1,0,1


In [40]:
sample_test['model_label'].value_counts()

0    8347
1    6280
Name: model_label, dtype: int64

In [41]:
# 将jd中的月薪转成年薪
def multiply_cols_fun(series, col1, col2):
    col1_value = series[col1]
    col2_value = series[col2]
    return col1_value * col2_value

sample_test['min_annual_salary'] = sample_test.apply(multiply_cols_fun, axis=1, args=('min_month_salary', 'pay_months'))
sample_test['max_annual_salary'] = sample_test.apply(multiply_cols_fun, axis=1, args=('max_month_salary', 'pay_months'))

In [42]:
# 先把workyear_range_jd转成列表
def school_lables_to_list(series, location_col):
    location_labels = series[location_col]
    location_labels_list = [item.strip('\'') for item in location_labels.strip('[]').replace(' ', '').split(',')]
    first = int(location_labels_list[0])
    second = int(location_labels_list[1])
    return [first, second]

sample_test['workyear_range_jd_list'] = sample_test.apply(school_lables_to_list, axis=1, args=('workyear_range_jd', ))

sample_test[['workyear_range_jd', 'workyear_range_jd_list']]

,workyear_range_jd,workyear_range_jd_list
0,"[5, 10]","[5, 10]"
1,"[0, 1]","[0, 1]"
2,"[3, 5]","[3, 5]"
3,"[0, 1]","[0, 1]"
4,"[3, 5]","[3, 5]"
...,...,...
14622,"[10, 100]","[10, 100]"
14623,"[5, 10]","[5, 10]"
14624,"[3, 5]","[3, 5]"
14625,"[0, 1]","[0, 1]"


In [43]:
# work year match
def workYear_match_fun(series, workyear_range_jd_list, workYear):
    workyear_range = series[workyear_range_jd_list]
    workYear = series[workYear]
    if workyear_range[0] <= workYear <= workyear_range[1]:
        return 1
    else:
        return 0
    
sample_test['workyear_match_cvjd'] = sample_test.apply(workYear_match_fun, axis=1, args=('workyear_range_jd_list', 'workYear'))

In [44]:
def keyU_label_fun(series, school_labels_cv_list):
    school_labels = series[school_labels_cv_list]
    if school_labels:
        return 1
    else:
        return 0
sample_test['keyU'] = sample_test.apply(keyU_label_fun, axis=1, args=('school_labels_cv_list',))

In [45]:
# 数据ok
sample_test[['school_labels_cv_list', 'keyU', 'school_labels_jd_list']]

,school_labels_cv_list,keyU,school_labels_jd_list
0,[双一流],1,[双一流]
1,[],0,[]
2,[双一流],1,[]
3,[],0,[]
4,"[211, 双一流]",1,"[211, 985, qs100]"
...,...,...,...
14622,"[985, 211, 双一流]",1,[]
14623,[],0,[]
14624,[],0,[]
14625,[],0,[]


In [46]:
def school_lables_to_list(series, school_col):
    school_labels = series[school_col]
    school_labels_list = [float(item) for item in school_labels.strip('[]').replace(' ', '').split(',')]
    if school_labels_list[0] == '':
        return []
    return school_labels_list
sample_test['description_tfidf_pca'] = sample_test.apply(school_lables_to_list, axis=1, args=('description_tfidf_pca', ))
sample_test['position_tfidf_pca'] = sample_test.apply(school_lables_to_list, axis=1, args=('position_tfidf_pca', ))
sample_test['requirement_tfidf_pca'] = sample_test.apply(school_lables_to_list, axis=1, args=('requirement_tfidf_pca', ))
sample_test['kills_tfidf_pca'] = sample_test.apply(school_lables_to_list, axis=1, args=('kills_tfidf_pca', ))
sample_test['title_category_tags_tfidf_pca'] = sample_test.apply(school_lables_to_list, axis=1, args=('title_category_tags_tfidf_pca', ))

In [47]:
sample_test.shape

(14627, 42)

In [48]:
# 几个list如何处理
# parsed_currentSalary特征不使用
# min_month_salary和max_month_salary分别乘以pay_months，得到年薪数据，以k为单位。已处理
# salary特征可信度不高，清华模型就是简单的输入deep侧，除此以外没有任何处理。
COLUMNS = ['age', 'recruit_number', 'workYear', 'min_annual_salary', 'max_annual_salary', 
            'degree_index_cv', 'degree_index_jd', 'gender_index_cv', 'job_intentstatus_cv',
            'description_tfidf_pca', 'position_tfidf_pca', 'requirement_tfidf_pca', 'kills_tfidf_pca', 'title_category_tags_tfidf_pca',
            'work_duration_mean', 'equal_job', 'parsed_desiredSalary', 'school_intersect_cvjd', 'location_intersect_cvjd',
            'degree_match_cvjd', 'workyear_match_cvjd', 'keyU']

LABEL_COLUMN = "model_label"

# 离散特征
CATEGORICAL_COLUMNS = ['degree_index_cv', 'degree_index_jd', 'gender_index_cv', 'job_intentstatus_cv', 'school_intersect_cvjd',
                        'location_intersect_cvjd', 'degree_match_cvjd', 'workyear_match_cvjd', 'keyU']

# 文本向量特征
ARRAY_COLUMNS = ['description_tfidf_pca', 'position_tfidf_pca', 'requirement_tfidf_pca', 'kills_tfidf_pca', 'title_category_tags_tfidf_pca']

# 连续特征
CONTINUOUS_COLUMNS = ['age', 'recruit_number', 'workYear', 'min_annual_salary', 'max_annual_salary', 
                    'work_duration_mean', 'equal_job', 'parsed_desiredSalary', ]


In [49]:
COLUMNS.sort()
print(COLUMNS)
print(len(COLUMNS))

['age', 'degree_index_cv', 'degree_index_jd', 'degree_match_cvjd', 'description_tfidf_pca', 'equal_job', 'gender_index_cv', 'job_intentstatus_cv', 'keyU', 'kills_tfidf_pca', 'location_intersect_cvjd', 'max_annual_salary', 'min_annual_salary', 'parsed_desiredSalary', 'position_tfidf_pca', 'recruit_number', 'requirement_tfidf_pca', 'school_intersect_cvjd', 'title_category_tags_tfidf_pca', 'workYear', 'work_duration_mean', 'workyear_match_cvjd']
22


In [50]:
all_columns = ['cv_id', 'jd_id', 'model_label'] + COLUMNS
print(all_columns)
print(len(all_columns))

['cv_id', 'jd_id', 'model_label', 'age', 'degree_index_cv', 'degree_index_jd', 'degree_match_cvjd', 'description_tfidf_pca', 'equal_job', 'gender_index_cv', 'job_intentstatus_cv', 'keyU', 'kills_tfidf_pca', 'location_intersect_cvjd', 'max_annual_salary', 'min_annual_salary', 'parsed_desiredSalary', 'position_tfidf_pca', 'recruit_number', 'requirement_tfidf_pca', 'school_intersect_cvjd', 'title_category_tags_tfidf_pca', 'workYear', 'work_duration_mean', 'workyear_match_cvjd']
25


In [ ]:
sample_test[all_columns].to_pickle('../data_20220831/sample_test_processed_processed.pkl')